To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-20 05:25:06 __init__.py:190] Automatically detected platform cuda.


Load up `Llama 3.1 8B Instruct`, and set parameters

In [3]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.59%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.61 GB. Also swap space = 2 GB.
WARNING 02-20 05:27:12 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-20 05:27:26 config.py:542] This model supports multiple tasks: {'classify', 'embed', 'generate', 'score', 'rewar

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

INFO 02-20 05:30:22 model_runner.py:1562] Graph capturing finished in 39 secs, took 0.59 GiB
INFO 02-20 05:30:22 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 53.77 seconds


tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [4]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = [
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write an engaging and informative blog post about common anti-productivity pitfalls that people fall into without realizing it. The post should highlight at least five key pitfalls, such as lack of planning, procrastination, social media distractions, over-reliance on productivity techniques, and failure to track progress. Each section should include a real-world example, explain why it's a problem, and offer practical solutions to overcome it. End the post with a motivational conclusion emphasizing the importance of working smart rather than just working hard."
            },
            {
                "role": "assistant",
                "content": "Have you ever found yourself going through the day feeling like you've been busy, but not productive? Do you constantly feel overwhelmed with tasks and responsibilities, yet still unable to achieve your goals?\n\nIf so, then it's likely that you have fallen into one or more anti-productivity pitfalls without even realizing it.\n\nAnti-productivity pitfalls are subtle traps that sap our energy, dilute our focus, and ultimately inhibit our progress toward our goals. They are often overlooked yet have a significant impact on our productivity.\n\nThis post is here to shed light on these hidden barriers that slow us down. Along with learning about certain anti-productivity pitfalls, we will explore ways to deal with them effectively.\n\nUltimately, the objective is not just to work hard but to work smart and make every minute count towards your success.\n\n5 Anti-Productivity Pitfalls To Look Out For\nAnti-productivity pitfalls are behaviors that can lead to a decrease in productivity and overall performance.\n\nBy familiarizing ourselves with these pitfalls, you can take proactive steps to avoid them and stay on track toward your goals. So let's take a look at some common anti-productivity pitfalls that you should watch out for:\n\nRunning a Day Without Planning\nIt's easy to get caught up in the busyness of our day-to-day lives without taking a moment to plan out our tasks and priorities.\n\nWithout a clear plan, we often end up wasting time on less important tasks or getting overwhelmed by the sheer amount of work that needs to be done.\n\nSuppose you're a marketing manager, and you start your day by checking emails, social media accounts, and voicemails. Before you know it, half the day is gone, and you haven't even started working on your actual projects like creating new campaigns or analyzing data.\n\nSometimes, people even get stuck in this loop where they spend most of their time planning and organizing their work instead of actually doing it.\n\nFocusing On Prevention Instead Of Creation\nWhile preventing mistakes and errors is essential, solely focusing on avoiding them can result in a lack of progress and innovation.\n\nThis often leads to perfectionism, where people spend excessive amounts of time perfecting every single detail instead of meeting deadlines and moving forward with their work.\n\nFor example, you're writing a blog post. And instead of focusing on creating valuable content, you're obsessing over finding the perfect image or editing every word and sentence to make it flawless.\n\nYou'll be simply wasting your time on something that doesn't contribute to the overall value of your work.\n\nYes, it's important to pay attention to details and ensure quality, but it shouldn't come at the cost of sacrificing productivity or the actual work.\n\nProcrastinating Too Much\nProcrastination is a common anti-productivity pitfall that affects many of us.\n\nIt's the act of delaying or postponing tasks, even though we know it will have negative consequences in the long run. Letting things build up and pushing them off until the last minute can lead to stress, burnout, and a decrease in productivity.\n\nYou're a freelance graphic designer, and you have a project due in two weeks. But instead of starting it right away, you keep putting it off because you don't feel motivated or inspired at the moment.\n\nBefore you know it, the deadline is looming over your head, and you're scrambling to finish the project, causing unnecessary stress and pressure.\n\nProcrastinating isn't wrong. It often happens when we feel overwhelmed by a task or lack the motivation to complete it. But it's important to recognize when we're using procrastination as an excuse and learn how to overcome it.\n\nGetting Obsessed With Social Media\nWith every notification and scroll, we're pulled away from our work and into the endless black hole of social media.\n\nChecking our social media accounts too often, especially during work hours, can be highly detrimental to productivity. It's a distraction that eats up precious time and hinders us from focusing on our actual tasks.\n\nOne ping, and before we know it, we've spent half an hour scrolling through our feeds and watching videos instead of working on that report.\n\nIt's important to set boundaries for yourself when using social media during work hours. If you are struggling with creating boundaries, then take the help of effective apps like FocusMe, and Freedom to cut down on your social media usage during work hours.\n\nTaking Pomodoros Too Seriously\nPomodoros is one of the famous productivity increase techniques where you work for a set amount of time, typically 25 minutes, and then take a short break before starting another Pomodoro.\n\nThis method is definitely good, but sometimes, people can take this concept too seriously and end up taking more breaks than necessary. Or they may hyperfocus on the tool instead of the task.\n\nAnd that ultimately results in a decrease in overall productivity, as the breaks can disrupt your flow and make it harder to get back into work mode.\n\nIt's important to find a balance with the Pomodoro technique and use it as a guide, rather than a strict rule. Don't be afraid to adjust the timings or take longer breaks if needed.\nThis goes for any other time-management hack or tool out there. They should not be crutches but aids only.\n\nNot Keeping Track of Progress\nWithout tracking your progress, it's easy to fall into the trap of feeling like you're not accomplishing anything, which can lead to demotivation and a decrease in productivity.\n\nKeeping track of what we've accomplished, even if it's just small tasks, can help us stay motivated and see how far we've come.\n\nAnd through review, you also see where you need improvement and work on them specifically. Rather than focusing on something that you\u2019re already good at. Tracking progress can also give a sense of satisfaction and help maintain momentum toward your goals.\n\nSo, make sure to keep track of your progress, whether it's through a planner, project management systems, or analytics tools. It will help you stay on top of your tasks and\n\nWays to Deal with Anti-Productivity Pitfalls\nNow that we've identified some common anti-productivity pitfalls, it's essential to address how to deal with them effectively.\n\nCreate a daily plan and stick to it: Take the time at the beginning of each day to prioritize your tasks and create a schedule for yourself. Stick to this plan as much as possible throughout the day to avoid getting sidetracked.\nSet realistic goals and deadlines: Don't set yourself up for failure by creating unrealistic goals or tight deadlines. This will only add unnecessary pressure and stress. Instead, learn to set realistic, achievable goals.\nTake breaks when needed: While it's essential to stay focused, it's also crucial to take breaks when feeling burnt out or overwhelmed. These breaks can help you recharge and come back to your work with renewed energy.\nUse productivity tools: There are various tools, such as time-tracking apps, project management systems, and website blockers, that can help you stay on track and avoid distractions.\nEliminate distractions: Identify your biggest sources of distractions and find ways to eliminate or minimize them. This could mean turning off notifications or working in a quiet space.\nStay accountable: Find an accountability partner or system to help you stay on track with your goals. This could be a friend, colleague, or even monitoring tool that tracks your progress.\nConclusion\nProductivity can't be measured by the number of hours you spend working, but rather by the quality and efficiency of your work.\n\nAnd these efficient work hours often decrease with the above-discussed anti-productivity pitfalls.\n\nSo, it's crucial to recognize these pitfalls and take steps to avoid them for a more productive and fulfilling work life.\n\nBecause at the end of the day, it's about finding a balance between work and life, and avoiding these pitfalls can help us achieve that balance.\n\nIt might not be easy, but with determination and consistency, you can overcome these anti-productivity habits and become more productive individuals. So, let's start today and take the first step towards a more efficient and fulfilling work life."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Create a guide for solo founders on how to efficiently manage their workload. Discuss the challenges of running a one-person business, the importance of structuring tasks, and strategies such as batching, day theming, and prioritization. Provide a step-by-step approach to organizing recurring tasks, reducing context switching, and improving productivity. Conclude with actionable advice for maintaining focus and avoiding burnout."
            },
            {
                "role": "assistant",
                "content": "How to handle all your tasks\neffectively as a solo founder\nWhen we think of optimizing our business, we probably think about streamlining our systems and utilizing no-code, automation and AI, all of which are important, but we often overlook the biggest asset to our entire business which is ourself.\n\nWhen you\u2019re running a 1-person business, this is even more true as you are the engine behind all the systems that you have setup, and just like with cars, an efficient engine leads to much better performance. If you don\u2019t learn to optimize your time, you will never be able to reduce your weekly working hours and will find yourself spinning plates and juggling simultaneously like some sort of crazed circus performer. If you\u2019ve ever launched a business before, I\u2019m sure you\u2019ll know the feeling.\n\nUnfortunately most people will not think about this, and never get their working hours below 40+ hours per week. But by being strategic with your time, you can reduce your working hours, whilst maintaining (or even increasing) revenue.\n\nimage_4\n\nDon\u2019t try to do everything everyday\nYou can\u2019t do every task required for your business on a daily basis. It just doesn\u2019t work, but most people (myself included) fell into these traps when they start their business:\n\nThey jump around between every \u201cdepartment\u201d every day\nThey don\u2019t have a clearly defined structure for their recurring tasks\nThey don\u2019t put parameters on their time and get easily distracted by incoming emails, social media etc.\nThey roll with the chaos too much and end up highly stressed\nThankfully, there are a few easy steps which can turn everything around for you. It can take a bit of practice to get them running smoothly, but once you do, you\u2019re in for a much much smoother ride.\n\nLet\u2019s jump in.\n\nStep 1: Move your recurring tasks to the highest possible cadence\nWhat I mean by this is that you want to think about the types of tasks that you need to do on a recurring basis. These could be things like emails, social media posts, administrative tasks, tracking stats, sales outreach etc. Write them all down on a piece of paper.\n\nOnce you\u2019ve done that, think about how often you NEED to do these tasks. Emphasis on the word \u201cneed\u201d here. You may find that some tasks that you currently do every day, could actually be done once a week, or perhaps even once a month.\n\nFor example, emails probably need to be responded to every day, so there\u2019s not much movement there.\n\nTracking stats on the other hand probably only needs to be done once a week, or maybe even once a month. Do you need to have enterprise-level daily tracking, or would a once-a-month overview be sufficient?\n\nAnother example is social media posts. Many people write social media posts every day, whereas they could easily be written once a week and scheduled for the entire week. If you\u2019re writing social media posts every day, you\u2019re doing a task 5 times a week, that you could do just once, albeit for a longer time. What\u2019s the difference I hear you ask? You want to avoid context switching in your day as much as possible. If you have to write 2 social media posts every day, it will likely take you 30 mins to do that, for high quality posts. If you do that 5 days a week, that\u2019s 2.5 hours of social media writing every week. Alternatively, if you did it only once a week and created all 10 posts in one go, you will likely find it takes you 1.5-2 hours. Why? Because it takes time to get into the rhythm of writing. Much like when you start pedaling a bike, the first few pedals are tough until you build up momentum. The same is true with our brains. When you put the social media content creation all on the same day, you cut out the tough beginning part each day, and end up reducing your time.\n\nWhich leads me to Batches\u2026\n\nStep 2: Organize your tasks into batches\nBatching is not a phrase I coined, but is a great way to think about your tasks. Batching is the process of taking all tasks of a similar nature and putting them together. Looking at the social media task above, this is a creative writing task so is also similar to writing blog posts for example. We can take both of these tasks and add them to a \u201cCreative writing\u201d batch. We take the momentum that we already started with the social media writing and we continue it over to the blog writing.\n\nDifferent types of tasks require a different way of thinking and constantly switching between them is jarring for our brains. Using the car analogy it would be like changing gears from 1 to 5. Not easy. Tasks of a logistical nature like event planning, admin, calendar scheduling etc. are all a very different pace than creative writing. The trick is to identify the tasks that go well together and put them into batches.\n\nHere\u2019s an example of a typical founder\u2019s day:\n\nfounders_day\n\nAnd here are those tasks moved into batches:\n\nbatches\n\nStep 3: Delegate your batches to specific days\nNow that we have our batches, we take it a step further and apply them to specific days. We might have multiple batches on a day, or we might just have one. If you do have multiple batches, again try and place similar types of batches together. Ideally you would have just one batch a day. For me, it looks something like this:\n\nday_theming\n\nAny time an email comes in or new task pops up, I just schedule it for the next day when I\u2019m dealing with that type of task. e.g. Any admin tasks that come up during the week, I\u2019ll snooze until my Admin day (Friday) unless it\u2019s urgent and then I deal with them all in one go.\n\nStep 4: Do the quickest task first\nFinally, now that you have your batches setup and assigned to specific days and you\u2019re delegating your tasks to each day accordingly, your business should be running smoothly and you are ensuring that everything gets taken care of every week, without jumping around like a trampolinist.\n\nYou can now add a final touch of finesse to your productivity machine by organizing your tasks in a way which allows you to move through them more fluidly. An easy trick here is to simply organize your tasks by how much time each one is likely to take. For example:\n\ndo the 5-min task first\nthen the 15-min task\nthen the 1-hour task\nand finally the 3-hour task\nThe reason for this is 2-fold:\n\nIt can sometimes be hard to get started, and by doing the easiest thing first, you jumpstart the process and gradually build up momentum. Then once you\u2019ve built up momentum, it\u2019s much easier to blast through the bigger tasks. In the words of Isaac Newton \u201cAn object that is in motion will tend to stay in motion\u201d. Make it easy on yourself and start small.\n\nIn the example above, you would have completed 2/4 tasks within 20 minutes which psychologically feels like you\u2019re halfway through. Checking things off your to-do list in quick succession, gives little dopamine hits which rewards you and makes you much more likely to continue at the same place, plus it feels great.\n\nThat\u2019s all for this week. I write these guides every week. If you found this useful, you can subscribe here to get the next guide in your inbox."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a case-study-style blog post featuring multiple indie founders who successfully manage multiple products. Include their motivations, strategies for balancing work, key challenges, and insights on staying productive. Cover topics like prioritization, automation, delegation, and revenue diversification. Include real quotes and actionable tips to help other founders learn from their experiences."
            },
            {
                "role": "assistant",
                "content": "Here's how these founders are\nbuilding multiple products at once\nby James Fleischmann\n\nI'm always blown away by folks who can juggle multiple products.\n\nSucceeding with one product is hard enough. But they're succeeding with a portfolio and making it look easy.\n\nI spoke with some of them to understand how they do it. Here's what they had to say.\ud83d\udc47\n\nWhy indie hackers are working on multiple projects\nTony Dinh of DevUtils, Xnapper, and Typing Mind:\n\nI want to use my own products so that I can make them perfectly fit my workflow. I have a lot of ideas that I would build for myself, but I only build ones that have the potential to also get paying customers.\n\nIt\u2019s the sweet spot between \u201cwhat I want\u201d and \u201cwhat people want\u201d that allows me to build for myself and sell to other people.\n\nAnother thing that inspired me to work on multiple products: diverse income streams. If one product fails, by the market or by accident (you never know), I will still have other products that bring in revenue.\n\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nA lot of the time I'm scratching my own itch.\n\nAnother reason I might start a project is to learn something new. It feels amazing diving into the unknown.\n\nI also totally suffer from shiny object syndrome. I'm bad. I'll see a slight gap or a poorly curated popular site, and I'll lose hours on domain searches while daydreaming of what I could cook up.\n\nWhen I focus on 1 thing, I see better results. But I love dabbling and every day is different and challenging.\n\nFelix Wong of Zlides, Abstract Lab, and Ventureslist:\n\nI treat them as fun things to do. They're opportunities for me to experiment with new things. They're excuses for me to build things that are completely under my control.\n\nI already have experience in founding and selling a startup, so I don't have ambitious goals for them. And I don't ever want my projects to replace my work.\n\nJavier Velazquez of Formwise, White Label ChatGPT3, Brandlyft, and LeadDragon:\n\nMy agency had a great network and reputation in the agency and marketing worlds, so building a SaaS allowed me to cross-sell the product to my colleagues and build another revenue channel.\n\nJohn Doherty of Credo and Editor Ninja:\n\nQuite honestly, I was feeling super unmotivated on Credo and wanted something new to sink my teeth into. It's been fun to get my hands dirty on something that doesn't have technical debt.\n\nStarting a second business has been all about reclaiming my energy and my zest for shipping new things. It's also to see if I'd want to work on this specific business in a more full-time capacity in the future.\n\nThe upsides vs the downsides\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nUpside: Continuous motivation and fun.\n\nDownside: Seeing slower results.\n\nJohn Doherty of Credo and Editor Ninja:\n\nI like that my days are varied. I get to work on both managerial-level things at one business and deep in the weeds on the other.\n\nBut that's also the hardest part. My day is varied and there is a lot to do!\n\nI also find the discrepancy between one larger business and one smaller business to be mentally challenging, as I can't expect out of the smaller brand what I expect out of the bigger one around growth, team, maturity of offer, customers, and so on.\n\nFelix Wong of Zlides, Abstract Lab, and Ventureslist:\n\nI like building and running multiple things because it allows me to grow and fail faster in many aspects. And I love the flexibility \u2014 jump from one side project to another, anytime. Stay hungry, stay foolish!\n\nBut the worst part is that it's itchy. I always wanna touch my projects.\n\nTony Dinh of DevUtils, Xnapper, and Typing Mind:\n\nThe best part is multiple income streams.\n\nThe worst part is that the management can become very messy, real quick if you are not careful. Mixed revenue reports, shared accounts, email support, partnerships, multiple payment accounts, etc. \u2014 it took me quite a while to find a workflow to consolidate all this and I still have to improve it day by day.\n\nHow to find the right ideas (plural)\n\ud83d\udca1 Tip: Choose (mostly) low-effort ideas\nTony Dinh of DevUtils, Xnapper, and Typing Mind:\n\nBuild products that require low maintenance effort.\n\nSaaS products are very needy and require constant attention \u2014 customer support, keeping the system up, etc. I don\u2019t think I could handle many SaaS at once.\n\nDownloadable apps are much more relaxed, with no server to take care of. There's only customer support. For me, most customers\u2019 emails are feature requests and require no action. So it\u2019s quite good.\n\nInfo products are even better, with almost nothing to maintain. I don\u2019t have any info products at the moment, but I\u2019m looking to add this to my income stream in the future.\n\n\ud83d\udca1 Tip: Choose ideas that solve your own problems\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nWork on things that solve your problems, you're curious about, or that level up your skills. This means your motivation is unlimited, meaning you can find these hours in your day. Client work is often a struggle. Passion projects are not.\n\n\ud83d\udca1 Tip: Choose ideas that focus on the same customer\nJohn Doherty of Credo and Editor Ninja:\n\nIt is easier to run multiple companies when they both serve the same customer. We serve digital agencies at both companies. People like Pieter Levels serve remote workers.\n\nPick one customer and build multiple things for them.\n\n\ud83d\udca1 Tip: Follow your enthusiasm\nMatthew Gordon of Burndown:\n\nThe thing that kills more products in their infancy than anything else is the death of enthusiasm. Genuine enthusiasm is a limited resource. When it dies, the rest of the dominoes often fall like a house of cards (so to speak). Estimate your interest. Estimate the work. Estimate the work timeline. Do your estimates add up? Source\n\n\ud83d\udca1 Tip: Get soft validation\nTony Dinh of DevUtils, Xnapper, and Typing Mind:\n\nThese days, I have a decent-sized audience on Twitter, so I can use that advantage to validate ideas. I usually tweet about the idea or build a small prototype/MVP to see if people are interested. It was definitely harder in the early days when I didn't have an audience.\n\nHow to stay effective on multiple projects\n\ud83d\udca1 Tip: Break everything into small tasks\nFelix Wong of Zlides, Abstract Lab, and Ventureslist:\n\nI break down my roadmap and goals so that they're as small as possible. This makes it so I can see my progress and bring things together gradually. For example, if I need to publish four articles a month, I break it down into one paragraph per day, and I convert it into a routine. You don't have to force yourself to write a whole article in a day.\n\n\ud83d\udca1 Tip: Prioritize\nFelix Wong of Zlides, Abstract Lab, and Ventureslist:\n\nI decide what to work on according to the ICE scoring system:\n\nI = Impact\nC = Confidence\nE = Ease\nI take a task and give it a score for each of these attributes from 1-5, then I multiply them together. The highest number is the one that I complete first.\n\n\ud83d\udca1 Tip: Focus on one task\nTony Dinh of DevUtils, Xnapper, and Typing Mind:\n\nI break 90% of my tasks into small chunks of work that can be done within one working session (about 3-4 hours). This includes building, testing, deploying, and writing docs.\n\nBecause of that, I can be fully focused on one task of one product on one specific day.\n\nAnd on a typical working day, I only do one task. Then, I spend the rest of the day on marketing, customer support, and interacting with the community (mostly on Twitter).\n\nOther days, I do smaller tasks with a lot of context-switching, which I find quite fun and relaxing.\n\n\ud83d\udca1 Tip: Automate (and the \"Rule of two\")\nFelix Wong of Zlides, Abstract Lab, and Ventureslist:\n\nI have a few automated workflows to help me clear out tedious tasks. It allows me to utilize the little bit of time that I\u2019d otherwise spend on them.\n\nFor example, I onboard paying customers through Email Octopus and map all email addresses to a CRM on Airtable for engagement drip and upselling. These workflows are usually done with Zapier.\n\nTony Dinh of DevUtils, Xnapper, and Typing Mind:\n\nI try to automate everything that can possibly be automated. Especially around customer support.\n\nSelf-refund, no questions asked\nA lot of FAQs\nAdd tips and GIFs around the app UI\n\"Rule of two\": When the 2nd customer asks for something, I'll try to automate it.\n\ud83d\udca1 Tip: Timeblock your projects\nJohn Doherty of Credo and Editor Ninja:\n\nMy Mondays are mostly dedicated to internal Credo stuff. Tuesdays and Wednesdays are for external calls for either Credo or EditorNinja. Thursday is mostly for EditorNinja. And Fridays (if I work them), are varied depending on what the week held for me to that point.\n\nI also block off 4-5pm every Monday through Thursday to just focus on EditorNinja.\n\n\ud83d\udca1 Tip: Focus on the project that pays the bills\nJohn Doherty of Credo and Editor Ninja:\n\nRunning multiple companies all comes down to time and energy management.\n\nFocus on the one that brings in the most revenue until you have a team mostly running that business, then you can put more time into the smaller one.\n\n\ud83d\udca1 Tip: Dogfood your product\nTony Dinh of DevUtils, Xnapper, and Typing Mind:\n\nI use all of my products every day.\n\nIt's the #1 source of motivation to continue working on the products, or just to push out a new update.\n\nAlso, because of this, I often find bugs before the customers do, so that's even better.\n\n\ud83d\udca1 Tip: Hire a virtual assistant\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nI hired someone to help with admin 4 days a week.\n\n\ud83d\udca1 Tip: Leverage tools to stay organized\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nAfter years of using Trello I'm now all-in on Notion. I have one team account for Love Curated shared with one other person. Then I have another team account with my wife to keep track of home loans, finances, upcoming trips, etc. I dig being organized.\n\nThen iA Writer for light lists/writing (sync'd with Dropbox). Clear app (think I bought it 8 years ago for $5) on the iPhone for shopping and temporary lists.\n\nMy desktop icons often act as a to-do list too and I'm always trying to clear those out.\n\n\ud83d\udca1 Tip: Create a publishing schedule\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nIt's super difficult to shift attention so many times a day. Without a schedule, I can blink and I haven't posted on Email Love in 2 weeks.\n\nA huge breakthrough this year, due to hiring, is having a publishing schedule in Notion. We now aim to post 1 bit of content on One Page Love every weekday.\n\n\ud83d\udca1 Tip: Use one product to grow the other\nJohn Doherty of Credo and Editor Ninja:\n\nWe have a weekly agency email within which we often promote EditorNinja.\n\nSo, EditorNinja has benefited from Credo, but not vice versa.\n\n\ud83d\udca1 Tip: Create a holding company\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nBecause of earnings increasing over the past couple of years, I was forced into VAT land here in South Africa. So I decided to wrap all my side projects into one parent brand called Love Curated, which is now a Pty Ltd. I am CEO and 100% shareholder, which doesn't mean much, to be honest.\n\nJavier Velazquez of Formwise, White Label ChatGPT3, Brandlyft, and LeadDragon:\n\nI have a holding company for ease of doing my taxes since I have multiple businesses that I have equity / interest in. My company is called Hyrule Capital\n\n\ud83d\udca1 Tip: Work your buns off\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nYou simply need to work damn hard and push. I'm not afraid to put in the hours, feel a side product feature out, then totally bin it. I've learned I can't make decisions properly without dabbling. Talking/thinking about ideas gets me nowhere.\n\nHow to survive the hustle\n\ud83d\udca1 Tip: Take breaks (and naps)\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nI start working around 7am and finish up at 7pm every day. But I take tons of breaks to surf and run. I also nap for 18 mins every day around 3pm.\n\nWhy an 18-minute nap? Man, I\u2019ve been trying to perfect this strategy for years. So, 20+ has a longer wake time and at 30+ you are almost grumpy having to wake up. An 18-minute afternoon nap, followed by an espresso, banks me easy 2+ hours of super productive screen time.\n\n\ud83d\udca1 Tip: Find what keeps you going\nFelix Wong of Zlides, Abstract Lab, and Ventureslist:\n\nCoffee, sports, and family time are where I find my relief.\n\n\ud83d\udca1 Tip: Say \"no\"\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nYou need to say \"no\" a lot more to work on your passion projects. Especially to things that make you money. I'm always playing the long game and mostly playing where my motivation lies. If I was after cash only I would simply conduct Enterprise Landing Page audits every day of the week, yuk.\n\n\ud83d\udca1 Tip: Clock out\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nI am learning to accept that side projects can never be finished.\n\nThere is unlimited optimization and marketing to do. The only true time it's done is when it's sold or the domain expires.\n\nSo most evenings I try to shut off, cook, and watch a series with my wife. I rarely work late anymore and it's awesome. I dig sleeping, ha. At the same time, my wife accepts my chosen path of building online and knows sometimes we need to focus on getting it done. No matter what.\n\n\ud83d\udca1 Tip: Ask yourself if building multiple products is right for you\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nIf you are learning and motivation is high, I would do it. If it's to try to make a quick buck, I personally wouldn't.\n\nJohn Doherty of Credo and Editor Ninja:\n\nThink hard about it, and make sure you are not doing it simply because you are bored, or because you're in the middle of something big and it feels hard. You don't want to be distracted and starve both of growth. It's better to push through that hard patch or figure out how to make your current thing less boring.\n\nOnly once you've stabilized that first business and have a team running it, including an operator with strong business ops experience, should you look to start another business. Looking back, I would've cemented having a strong operator and strategist at Credo before working on EditorNinja more.\n\nBut there's also never a good time to start something like a new business. There are just those of us who don't let that hold us back.\n\nEven with all these tips, is it sustainable?\nJohn Doherty of Credo and Editor Ninja:\n\nIt's not easy to do. It requires a lot of focus on each when you're working on them. You'll probably work more evenings than when just running one company.\n\nAt this point, I can continue running both indefinitely, but should EditorNinja or Credo really grow faster, then it wouldn't be sustainable long-term.\n\nIf it's not sustainable, the two options would really be to sell Credo or hire a strong operator. I'm open to both options.\n\nRob Hope of One Page Love, Email Love, UX Love, and Landing Page Hot Tips:\n\nI can honestly say every day at 7am I am frothing to start working on my network of sites and side projects. I have had this for 10 years and it hasn't gone away.\n\nThat all said, if my motivation died, I would probably go where new motivation led.\n\nTony Dinh of DevUtils, Xnapper, and Typing Mind:\n\nHonestly, I have no idea if this is sustainable \u2014 it\u2019s my first time, haha. As of now, I find working on multiple products fun and pretty easy.\n\nI have seen successful indie hackers who have one single focus, and I've seen successful indie hackers with multiple ones. Some who have multiple eventually drop the less successful products to focus on the one that gets the most traction.\n\nI have no idea if that will be me in the future, but let\u2019s see."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Create a practical guide on staying productive while working from home. Cover essential topics such as setting up a dedicated workspace, creating a structured schedule, setting boundaries, taking breaks, and using productivity tools. Provide actionable tips and personal anecdotes for maintaining work-life balance and avoiding distractions."
            },
            {
                "role": "assistant",
                "content": "How to Be Productive While Working\nFrom Home: Tips & Tricks Guide\nby Oscar Vicente Gonzalez Greco\nHey there! In these unprecedented times, we've all had to adapt to working from home and it's no secret that staying productive in a less structured environment can be tough. But with a few simple tweaks to your routine and mindset, being productive at home is not only possible, but it can even be enjoyable. In this guide, I will share with you some tips for being productive, which have worked for me, while working from home.\n\n1. Create a dedicated workspace\nFirst and foremost, it's important to create a dedicated workspace where you can focus on your work. Set up a comfortable working environment that separates work from home life. This could be a corner of your living room or a spare bedroom that you can convert into a home office.\n\nI personally have a dedicated room in my house that I use as a home office. It's best if you can lock it off from the rest of your home, so keep this area free from distractions\u2014and tidy!\n\n2. Stick to a schedule\nSticking to a schedule is one of the most important things when working from home. Plan out your day and dedicate specific hours to work. This will help you stay on track and avoid distractions.\n\nI recommend starting your day off with a few hours of work, followed by a break. You could take a nap or go for a walk outside. After your break, it's time to get back to work! This allows you to recharge and focus on your tasks at hand.\n\n3. Set clear boundaries\nWorking from home can quickly blur the line between work and personal life. Setting clear boundaries can help you maintain a work-life balance. When your workday ends, close your laptop and step away from your workspace.\n\nI like to wake up early and start working on my project Align, then after breakfast, I shower and start working on my regular 9 to 5. From 5 to 6, I workout; after that, I dedicate 100% to my family and don't dare to look back at work.\n\n4. Take breaks\nJust like in an office setting, taking breaks is crucial to staying productive. Take frequent breaks to stretch your legs, have a snack, or simply relax. This will help you stay energized and refreshed throughout the day.\n\n5. Use productivity apps\nTo make the most out of your workday, consider using productivity apps such as Structured, Sorted3, or, coming soon, my latest project, Align. These apps will help you stay organized and on task throughout the day. They can also remind you of important tasks that need to be completed and provide you with statistics about your productivity.\n\nIn conclusion, working from home requires careful planning, good time-management techniques, and discipline. Follow the tips we've laid out above, and remember to check out my new blog for even more productivity tips and tricks. And if you're interested in staying up-to-date with my developments, check out my latest project, Align. Let's maximize our productivity and conquer the workday together."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write an in-depth guide on how makers can apply design thinking to build and improve their products. Explain each stage of design thinking\u2014Empathize, Define, Ideate, Prototype, and Test\u2014with practical exercises and real-world applications. Include a case study of a maker who used this methodology to improve their product. Provide actionable steps for indie creators to iterate effectively and build user-centric solutions."
            },
            {
                "role": "assistant",
                "content": "Design thinking for makers\nby Daniela\nThere is no rulebook for makers, and that can be both empowering and terrifying at the same time. But a methodology called \"design thinking\" can make it easier to come up with ideas, test them quickly and iterate until you find the best solutions for your users.\n\nI\u2019m a UX designer and a maker, and in this post, I will walk you through an example of how I\u2019ve used design thinking to release v2.0 for my platform ctrlalt.cc in a relatively short amount of time (a couple weeks) doing everything from redesigning it to coding new functionality for it. I'll also give you a few creative exercises to help you along the way.\n\nWhat is Design Thinking?\nA generic definition for \u201cDesign thinking\u201d is \u201cA human-centered approach to innovation that draws from the designer\u2019s toolkit to integrate the needs of people, the possibilities of technology, and the requirements for business success\u201d - (Tim Brown from IDEO)\n\nI like to think of it as a zoomed out map that can help you reach and navigate a creative mind space on demand. It\u2019s a very flexible and non linear process that you can use at any stage in your project, regardless of its size or nature. Covering anything from new products to new features, new landing pages, design updates, creating courses, getting unstuck when you've run out of ideas on how to fix a problem and any other creative project that you can think of and addresses an audience.\n\nLet's dig into how it works.\n\n1. Empathise\nAs you\u2019ve guessed, this step is all about understanding who you are building for and what their problems are. In a standard model, this is where you would create user personas, conduct user interviews and collect as much knowledge about your users as possible. And don\u2019t get me wrong you can still do all those things if you want, but from a maker perspective, your resources are more limited, so you need to focus on what\u2019s gonna get you closer to your results in the most effective way.\n\nIn my case, from my chats with them, I know that my users are founders, makers, designers, and developers and that they want to discover tools that help them with their tasks, without wasting too much time. Some of them also want others to discover their projects. And they also want to get backlinks for their projects, and have people come to their website on a regular basis.\n\nYour situation will look different but defining your project based on the problems you are trying to solve works in the same way. Don\u2019t have any users yet? This still works. Think about who you are building for, go on a social media where the people that you are building for hang out, start talking to them and a percentage will want to help you out and answer a few questions, which is all you need right now! Start asking them about the problem you are trying to solve. See what solutions they currently use and find out what they don\u2019t like about them. And there you have it, your initial bit of insights into what your users want.\n\nNow that you have this information, it can start to look daunting again. You will have a lot of people telling you a lot of different things, so how do you know what to take forward? Enter step number 2: Define.\n\n2. Define\nFor this step, we basically use the info that we\u2019ve gathered and dive a bit deeper into understanding what problems this group of people share, to find what areas we should focus on.\n\n\ud83d\udca1 Tip: An exercise that I find handy at this point is writing these problems down on post-its (physical or digital ones - you can use something like Figjam if you want to go digital). Make sure you use 1 post-it per problem. Then start to group together the post-it\u2019s referring to similar issues. If you need a bit more structure you can organise them by the stage in the user journey the issue occurs at:\n\nAwareness\nConsideration\nPurchase\nRetention\nAdvocacy\nYou\u2019ll start to see some clusters forming, and that will help you identify the areas that you should be focusing on.\n\nIn my case the main focus area ended up being daily activities. It was something people enjoyed and I was offering too little of it. This was also highly linked to increasing daily visitors for the products on the website, so it sounded like a great direction to go in.\n\nNow that I know what area I needed to focus on, the next step is creating a solution as a response to this. Which brings us to step 3: Ideate.\n\n3. Ideate\nDesign thinking is very flexible, which means there are a lot of different exercises that you can do at this step, depending on what you\u2019d like to achieve. Most exercises usually focus on team brainstorming, so I like borrowing some from the \u201cdesign sprint\u201d framework as I find they are great at pushing you into a creative mindspace and they are better suited to makers.\n\nA. \"How might we\" exercise\nI like to start with an exercise called \u201chow might we\u201d (you can read more about it here) where you look at the problems from your focus cluster (from step 2) and reframe them into solution seeking questions, following this structure \u201chow might we\u201d + action + for user + so that + outcome.\n\nFor example, in my case a \u201chow might we\u201d question would be \u201chow might we focus more on daily activities for makers so that their products get the spotlight more often?\u201d or \u201chow might we focus on daily activities for site visitors so that they come back to the site more often?\u201d\n\nB. Find inspo\nNext, with these questions in mind, you should go and find inspiration for potential solutions. This can come from anywhere, but looking at how other products answer your questions is a great place to start! That being said, I would recommend looking at solutions in a different industry than your own as it helps you think outside the box and come up with more original ideas.\n\nC. \"The Crazy 8s\" exercise\nNow that your creative juices are flowing, it\u2019s time to sketch out some potential solutions. And this exercise called \u201cThe crazy 8s\u201d (part of a technique called \u201cThe 4 step sketch\u201d) is perfect for squeezing ideas out.\n\nFor this you\u2019ll ideally need some paper (but a digital whiteboard works too) and a timer. Get 8 boxes ready, either digitally or by folding the bit of paper into 8.\n\nNow start the timer and every minute you will make a new solution sketch. You don\u2019t have to finish it, just sketch an idea for 1 minute and then move into the next rectangle when the minute is up. Try to sketch something each time, even if some of them might feel hard to do. Keep going until the 8 minutes are up. You\u2019ll be amazed at how many ideas you came up with when this is done!\n\nAnd now that you have all these ideas, you can start putting them together and sketching the solution you\u2019ll be going forward with. Don\u2019t put too much pressure on this though, it doesn\u2019t have to be right. This process is all about testing ideas and iterating, so follow your gut, and sketch the solution out, we\u2019ll find out if it\u2019s right later on.\n\nIn my case, my core solution was creating a timeline of daily launches that would include new products as well as relaunching products from the directory. This would encourage makers to launch and relaunch their products as often as they wanted, which would also mean fresh daily content for the website, encouraging visitors to come back daily and discover all the new & relaunched products.\n\nSo with this sketched out, the next step is to make the solution a bit more tangible so that people can interact with it.\n\n4. Prototype\nThe prototype can be as minimal as you want it to be, but the important thing here is that your core solution is functional, while spending a minimal amount of time on it. In a traditional setup you\u2019d do this in a day and then test it. But again, these rules are not set in stone, so you should adapt them to your specific case.\n\nFor me because it was quite a big build, I gave it a couple weeks. But regardless of the size, or the stage of your project, you have to be very diligent in cutting off all the \u201cnice to haves\u201d and keeping the focus on the core functionality, so you can get it in front of users as soon as possible.\n\nIn my case, I already had so much functionality and features, that I had to go in the opposite direction and ended up binning a lot of the old site, keeping pretty much the database and some basic functionality. The first release for v2.0 needed to focus on the daily launches, so I had to put a lot of the other tangent functionalities in \u2018coming soon\u2019 (there are still some of these throughout the site if you go and visit, but I\u2019m slowly launching them one by one, taking them through this same process)\n\nWhich brings us to the last step of this non-linear process: Testing\n\n5. Test\nNow that you have a functional version of your idea, you have to get it in front of people and see what they think! So you can gather feedback and iterate where needed.\n\nYou might have to go back to step 2 and redefine the problems after you\u2019ve presented users with a solution that maybe is not quite right for them. Or maybe your solution is not as smooth as it could be, which would mean going back to step 3. This is all part of the process and you should do this as many times as needed! It will only make your product better.\n\nIn a standard setup you\u2019d want to conduct live user testing, and gather insights by watching your users actually try the product. But makers have to be more creative with their resources, so launching your project (ctrlalt.cc is a great spot for this), getting it in front of real people and gathering feedback through email and/or social media is all you need! And will be invaluable in identifying all sorts of things that you\u2019ve missed, things you didn\u2019t think were needed, future focus areas, potential feature ideas and so on.\n\nIt\u2019s not a linear process and you will go through this cycle over and over again. So don\u2019t overthink it and launch as soon and as often as you can! It will help you create better products that better serve the needs of your customers.\n\nBeing a maker is not easy, especially when there are so many things to consider at every step. But if you don\u2019t know where to start, what to focus on next or need a reliable structure that you can adjust with time and will help you create better things on demand, you should give design thinking a go! You will not regret it."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a comprehensive article on how startup founders can overcome distractions and improve focus. Identify common sources of distraction such as poor planning, multitasking, and procrastination. Introduce a three-tier system for maintaining focus, including defining key objectives, planning tasks effectively, and using productivity techniques such as time blocking and deep work. Provide examples and practical strategies for implementation."
            },
            {
                "role": "assistant",
                "content": "How to squash distractions and get\nthings done \u2014 a 3-tier approach\nby G\u00fcnter Richter\nStart-up founders face challenges on their journeys and whilst many of these challenges are external to the business, by and large, the most significant challenge founders face is the ability to focus and prioritise.\n\nThis lack of focus poses a real risk because not only does it hinder progress, but it can, in some instances, set the business backwards.\n\nIn this article, we will examine how founders get distracted, why this is the case, and how founders can fix it. Using this three-tier model, I have been able to effectively move from managing one or two concurrent projects to five or more concurrently, as well as running internal operations like recruitment, billing, and staff development.\n\nWhere do distractions come from?\nThere are as many sources of distraction as external stimuli or thoughts in a day. I have distilled these down into the top seven:\n\nPoor planning and prioritisation. It sounds cheesy; failing to plan is planning to fail. Not having a plan or priorities means that someone else will plan and prioritise for you and your business \u2014 competitors, customers, staff, etc. This could be your competitors, your staff, your prospects, or your customers (note: this doesn\u2019t mean you should ignore what your prospects and customers are saying). The added danger is that when you do have some time on your hands, you won\u2019t know what to do with it and end up spending it on trivial things.\nShiny object syndrome. Ooh, look at the butterflies! This is the classic distraction. There is something new that grabs our attention, and we need to have a play with it. It isn\u2019t terrific when the shiny object has zero to do with the goal we are trying to achieve.\nInstant gratification. Instead of thinking about a longer-term goal, as humans, we\u2019ll often take a quick win. Instant gratification manifests itself in situations where instead of investing time in marketing or perhaps writing a blog post, we\u2019ll build a new, unnecessary product feature because we can immediately see the results of our investment.\nFear and risk avoidance. Often, it is fear that holds us back. Fear of failure, fear of what people think, or fear of missing out. Also, as humans, we will rather avoid risk than aim for some reward even when the chances of receiving the reward are much higher than the risk coming to fruition.\nProcrastination. It is speculated that creative people are procrastinators, but things still need to get done at the end of the day. There are many theories on why we procrastinate. One of the more common theories is that we do it not because we don\u2019t like the task at hand but because we don\u2019t like the feeling or emotion related to that task.\nMultitasking. As much as we believe we are good at multitasking, we are not. This is why talking on your mobile phone whilst driving is not allowed.\nPersonal factors. These are typically things like low energy levels, lack of concentration, and poor health or fitness.\nHow can we fix this?\nWhilst there are a ton of tactical fixes, for example, internet blockers, we need a holistic planning system. I have developed a three-tier system that I use,:\n\nDefining your strategy and key objectives\nIdentifying and planning the tasks and activities you need to complete to meet your objectives\nUsing tips, methods, and tools to complete the tasks whilst remaining focussed\nSound obvious? Many people will do one or two of these, but all three are a must if you want to be genuinely successful. Let\u2019s examine why this is the case and how you can implement this system.\n\nStep 1 \u2013 Defining your strategy and key objectives.\nSo much can be asked about what a strategy is and how to define it. In short, a strategy is identifying where you want to be, where you are now, and the steps you\u2019ll take to get there.\n\nIn this step, you want to avoid vague statements, such as \u201cI want to build a business\u201d. Be more specific, \u201cI want to build a profitable business from multiple streams of annuity income in the next two years\u201d. Now you will take this down a level and identify the key objectives that will help you achieve this. Things like, \u201cI want to write and publish an eBook about productivity\u201d or \u201cI want to build a SaaS product that helps TV fans find their next series to binge\u201d.\n\nNow we\u2019ll take the SaaS example and break this down in the next step.\n\nStep 2 \u2013 Identifying and planning tasks and activities.\nHaving an objective of building a SaaS product is quite meaty, so we will need to break this down in more detail. Identify the main tasks you will need to complete to achieve this. Start at a high level and then go a bit deeper. These tasks might be identifying the target market, validating the idea, designing a minimum viable product, building the back-end, building the front-end, testing, launching the beta version, gathering feedback, tweaking the product, and fully launching. Break down each of these into more detail. Identifying the target market, defining research objectives, validating the market size, defining user personas, conducting interviews, identifying desires and pain points, formalising the product proposition, and defining the go-to-market strategy. When the next-level tasks still seem meaty, break these down a little more. The rule of thumb is, \u201cCan I easily understand the definition of 'done' and tick this item off\u201d.\n\nOnce this is done, set up a Kanban board and put all the tasks into the backlog. I use ClickUp for this, but many other tools are available too. As we get through the article, I\u2019ll talk a little more about why I use ClickUp.\n\nOnce your backlog is complete, consider your available time to work on this project. You may have a full-time job or other projects, or this may be a part-time endeavour. Try and estimate the effort required for each of these tasks. By now, you will undoubtedly have spotted that we are using some Agile principles here!\n\nConsidering the effort of each task and the time you have available, put in the actual date that you will work on each task. This is where a tool like ClickUp becomes very useful. Once you\u2019ve added your dates, you can add a Gantt (timeline) view. You can even add task dependencies that can help you plan in more detail. Now it is easy to switch between your Kanban and Gantt views. The Kanban view allows you to add tasks quickly, plan at a high level, and enjoy a dopamine hit when you drag an item across to the \u201cdone\u201d column. The Gantt view allows you to plan in more detail and understand how long your project might take.\n\nThe final bit to do is link ClickUp to your calendar. Now when you view your regular calendar and ClickUp calendar together (in the same way you would when booking a meeting with some), you can visualise when you should be working on your tasks and where there are potential conflicts, allowing you to replan if necessary. If your calendar is shared (i.e. colleagues can view availability and book meetings), then you should block off the time slots as reflected in your ClickUp overlay calendar. This is to protect your time.\n\nStep 3 \u2013 Staying Focused\nWith time protected in your calendar, you can get down to some \"deep work\", as author Cal Newport calls it. Now is the right time to haul out tools, techniques, tricks, and hacks!\n\nNotifications off\nA foundational trick is to switch off your notifications. Mail, Outlook, Slack, Teams, whatever. Switch them all off. Choose a few slots in the day to consciously open your email or messaging app and deal with items you\u2019ve received. Remember, email is the number one method for others to manage your work (instead of you managing it)!\n\nPomodoro technique\nGetting started, my number one method is the Pomodoro Technique, named after the inventor\u2019s tomato-shaped kitchen timer. The technique is simple, set a timer for 25 minutes and work exclusively on the task you\u2019ve booked out the time for. When the timer finishes, take a break for five minutes. Repeat this cycle another twice and then take a more extended break of 20 to 30 minutes. You can use any timer for this; I have an app which introduces a bit of analytics which is quite helpful.\n\nTime tracking\nIn addition to this, I record my time. Using ClickUp, you can start a time on each task. I have also used Toggle, which works very well. The value in this is that you can review what you have spent time on, analyse it, and use it to improve. Both in terms of optimising your time and your estimating abilities.\n\nBuddy system\nAnother tool to help you focus is accountability. Use a colleague, friend, or acquaintance. At the start of your focus session, you commit to what you will individually have completed by the end. This works face-to-face, where you might get together in a co-working space. You can do it over Zoom or Teams, and there is even a website that will hook you up with an accountability partner.\n\nYou will find things difficult at some point; we all do. However, if I find myself straying off course and thinking, ah, there is this thing I need to do, I ask myself, \u201cWill it make the boat go faster?\u201d. That is, will it help me achieve the goal that I am focussing on? The technique is from the gold medal Olympian Ben Hunt-Davis. You can read more in the book, Will It Make The Boat Go Faster?\n\nThere are many other approaches, some very specific to your work. For example, don\u2019t mess about formatting or spelling if you are writing content. Those can come later. If you are coding, set up your desktop(s) in a way that supports your tasks.\n\nThen, of course, there are the things that we should all be putting into practice but often don\u2019t. Get enough sleep, eat well, exercise, drink plenty of water, etcetera.\n\nFinally, some of the big no-no items. Do not try to multitask. During your focus periods, do not check social media. Not even for a minute; that\u2019s what the breaks are for.\n\nThis final section is not exhaustive regarding tools and techniques but hopefully has given you a starting point.\n\nWrapping up\nRemember, however, that these tools and techniques are not much good in isolation. You need a system like the three-step one I have laid out here.\n\nGood luck achieving the goals you have set, and feel free to reach out to me to talk about anything planning and productivity.\n\nUPDATE (19 January 2024). The feedback on this article has been so positive that I have done ahead and created a Notion template/dashboard to implement this system. You can check it out at"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write an insightful blog post categorizing different types of projects based on effort and obsession levels. Introduce the 50/50, 80/20, and 110% project framework. Explain each type with examples, discussing how people engage with projects based on motivation, efficiency, and passion. Provide insights on when to focus on each type for optimal productivity and fulfillment."
            },
            {
                "role": "assistant",
                "content": "Are your projects 110s,\n80/20s, or 50/50s?\nby Channing Allen\nEveryone I know who does great work obsesses over quality from time to time. And I mean really obsesses.\n\nFrom Walter Isaacson's Leonardo da Vinci:\n\nA peasant who worked in Vinci one day made a small shield of wood and asked Piero to take it to Florence and have it painted.\n\nPiero gave the task to Leonardo, who decided to create a terrifying image of a dragon-like monster breathing fire and belching poison. To make it naturalistic, he assembled parts from real lizards, crickets, snakes, butterflies, grasshoppers, and bats. \u201cHe labored over it so long that the stench of the dead animals was past bearing, but Leonardo did not notice it, so great was the love that he bore towards art,\u201d Vasari wrote.\n\nWhen Piero finally came to get it, he recoiled in shock from what in the dim light appeared at first to be a real monster. Piero decided to keep his son\u2019s creation and buy another shield for the peasant.\n\nThis dragon artwork represents what I call a \"110 project.\" I put projects [1] into three buckets: 50/50 projects, 80/20 projects, and 110 projects.\n\nA 50/50 project is one you engage based on how you happen to feel at a given moment. Watching Netflix, hanging with friends, scrolling social media, relaxing on a beach. It's mood-driven. If you're in the mood, you do it. If not, you don't. 50/50.\nIt's an 80/20 project if it follows the Pareto principle: you do the 20% of the work that will deliver 80% of the results. Usually this applies to projects you have to work on but don't want to work on. Or projects you think you want to work on but which mostly serve as means to the ends you truly want. Going to the gym to look good in a bathing suit. Reading a book to sound smart in front of others. Preparing TPS reports to avoid getting fired. And so on. When thinking about these projects, you'll often use terms like \"diminishing returns\" and speculate on the ROI on your time, energy, and attention.\nThis bean counting goes out the window with 110 (110%) projects. If 80/20 projects are about satisfying standards, 110 projects are about setting standards. Your central preoccupation becomes making the thing better, and a desire for \"efficiency\" only enters the equation when a deadline approaches or other demanding projects intrude on your attention. The zoomed-in view of working on a 110 project might look like a flow state, and the zoomed-out view might look more like \"passion\" or \"obsession\" or \"perfectionism.\"\nI was discussing this with a friend a few days ago, and she said: \"So basically it's a spectrum between play and work. 50/50 projects are play, 80/20 projects are normal work, and 110 projects are really hard work.\"\n\nThis felt off to me, but I couldn't put my finger on why. Then it hit me:\n\nProject Spectrum\n\nThe Adjacent Possible is where I share my most useful insights in bite-sized posts. One email per week. Many insights per email:"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a detailed article on how indie founders can save hundreds of hours per month using automation. Explain the benefits of automating repetitive tasks, provide real-world examples, and list specific workflows that can be optimized. Compare automation tools like Zapier and Make, and guide readers on how to get started with process automation. Conclude with a step-by-step method for identifying tasks to automate and setting up efficient workflows."
            },
            {
                "role": "assistant",
                "content": "How we save 400+ hours per\nmonth through automation\nby Jimmy Rose\nWhen you\u2019re running an indie software business, there are SO many things that need to be done. You either have to do them yourself, hire someone, or get software to do it for you.\n\nThere\u2019s only one of you. Hiring is difficult and sometimes prohibitively expensive, especially in the early days. That leaves software.\n\nIn our business, we save over 400 hours per month with automation. Here's how you can do the same.\n\nProcess automation \u2014 is it really worth it?\nYou might be wondering if it\u2019s worth the bother. After all, if you don't have any experience with workflow automation tools, it can be a pretty daunting thing to jump into. Plus, there\u2019s the time you spend setting up automations. If you spend an hour setting up an automation, will you ever get that back?\n\nIn my experience, if you automate the right things, the answer is an obvious yes.\n\nHere's an example. Say you have a form set up on your website to capture new leads. You get an email in your inbox with the lead\u2019s contact details, which you then have to manually input into your CRM. That might take a minute to do. That\u2019s pretty fast. Why bother automating it?\n\nIf it takes you an hour to create that automation, you\u2019ll get your initial hour back after it\u2019s processed 60 leads. If you\u2019re doing 5 leads a day, that\u2019s 12 days. And after that, you\u2019ve got a free 5 minutes every day. Doesn\u2019t sound like much, but the more processes you automate, the more they add up.\n\nAt the time of writing, our business automates around 50,000 tasks per month. If you assume 30 seconds per task, that\u2019s 416 hours per month. A fulltime employee is around 170 hours/month.\n\nIt\u2019s so much more than just time, though. Other benefits include:\n\nNo human error - automations don\u2019t make typos, copy data to the wrong fields, or miss a character or two when copying and pasting.\nScalability - as your business grows, the automations just run more often. You don\u2019t need to add people.\nHeadspace - when you don\u2019t have to switch tasks all the time, you\u2019re more productive. Plus, there\u2019s a nice feeling that comes with knowing things are always ticking along without you actually doing any work.\nWhat should I automate?\nThe most basic automation typically follows the formula: When this happens, do that. You can also add multiple actions or conditions.\n\nAn example that includes each of the elements would be: When someone fills out my screening questionnaire, post a message in Slack, and if they said their budget was more than $20000, send me an SMS to call them immediately.\n\nIf you\u2019re anything like me, your mind will be buzzing now with all the processes you could potentially automate. But this raises an important question: What exactly should l automate?\n\nA great place to look for inspiration is by simply exploring the Zapier or Make websites, looking at the automations and templates they have there, and seeing what looks useful.\n\nOnce you get into the mindset, you\u2019ll notice more and more processes that are ripe for automation. To keep track of these ideas, I recommend keeping an automation wishlist \u2014 a simple spreadsheet or Google doc that you add ideas to as they come up.\n\nTo help you get started, I'll break down the types of things that should be automated, then I'll give specific examples.\n\nWhat types of tasks to automate\nRepetitive tasks: Everything from manual data entry to onboarding new clients and scheduling social media posts. A typical example is sending the same or similar emails\nDouble-handling: Moving data from one place to another, especially multiple times. Like when a new client comes in and you have to add them to your CRM, invoicing, project management, and more. Double-handing and repetitive tasks typically go hand-in-hand\nInbox cloggers: Instead of getting bogged down with emails, you can automate support handling or get notifications on a slack channel.\nTasks that you forget: It\u2019s hard to keep on top of everything when you\u2019re running a business. Automation helps you get the information you need without having to go digging, or it can just remind you to do things that you might forget while you\u2019re buried in other tasks.\n\u200b\u200b### Specific examples for SaaS\nHopefully you already have plenty of ideas on what to automate. But if you\u2019re looking for more, here are some examples specifically for the SaaS founders out there.\n\nOnboarding & Customer Success\nFlag new ideal leads: When someone signs up for a trial, use Lead Score by Zapier to find how big their company is. If they are large, create a task to reach out to them.\nAdd customers on LinkedIn: For anyone that matches criteria, generate a LinkedIn search URL and send it to yourself in Slack. That makes it very easy to add them.\nIncentivized feedback: When someone fills out a feedback form, extend their trial. We use this to encourage people to tell us where they heard about us for an extra 7 days of trial.\nParty channel: Add new paid signups to a channel in your Slack. It\u2019s like a little celebration every time.\nDemo tracking: When someone books a demo, tag them in your CRM to make it easy to follow up.\nMonitor usage: One week after signup, generate a report of customer activity and attach it to the customer record in your CRM. We use Mixpanel for this.\nIdentify Power Users: Each month, calculate a \"health score\" for every customer based on their usage. If they're above a threshold, tag them in your CRM.\nSupport\nQuick actions: If you're using Intercom, create \"Quick Links\" that send information to a Zapier or Make webhook. We use this for common actions like extending a trial, or for quick access to their billing profile, CRM profile, etc.\nStart conversations for feedback: If someone fills out a feedback form or NPS score, create a new conversation automatically and request they leave a review. Use canned replies with personalization.\nStart conversations for cancellations and downgrades: Likewise, start a conversation when someone cancels. Write a personalized message requesting feedback.\nKeep your team in touch with customers: Post NPS, comments, feedback, and cancellation reasons to a feedback Slack channel.\nContact Sync\nHaving the right information in all your systems enables better, more personlised communication. It's not sexy, but it's important. A bulk of our automations fit into this category.\n\nBasic sync: When customer information changes, update them in your other systems. It helps to have a \"single source of truth\" that you use as your master list, and update your other systems from there.\n\nPlan updates: When a customer changes their plan or adds a credit card, update your CRM and support system.\n\nSync unsubscribes: If you have more than one system that sends email, sync unubscribes between them.\n\nMarketing\nBlog post promotion: When a new blog post goes live, create tasks to promote it. Also add them to a spreadsheet so you have a record of all your content. Same goes with support articles, case studies, and any other content you create.\nNewsletter creation: Use the Zapier Chrome Extension to flag good content and add it to upcoming newsletters. You could also feed in any new blog posts, support articles, or YouTube videos.\nBlog post planning: Use the OpenAI API to create a draft brief for any posts you want to create, and automatically save it as a Google Document. We use AirTable for this, and the Google Drive link is added back to the record once generated.\nOther\nKeep tabs on your business: Post a summary of project management system activity to yourself each day or week.\nJust remember: Don\u2019t try to take on too much at once, as it leads to overwhelm. Pick something small to start and don\u2019t think you have to do everything on this list by tomorrow.\n\nZapier vs Make \u2014 which one should I use?\nZapier and Make are the most popular tools for automation. I recommend learning both, but Zapier is perhaps the best place to start if you are new to process automation.\n\nTo keep things simple, let\u2019s look at a brief overview of the two tools.\n\nZapier\n\nRelatively simple to use\nIdeal for newbies\nTons of ready-made automations\nLimited options for building complex workflows\nA free option, with paid plans starting at $19.99 per month\nMake\n\nMore complicated to use\nEasier to create complex automations\nBetter suited to those with prior experience or a programming background\nA free option, with paid plans starting at $9 per month\nIn reality, there are tons of intricate differences between these two platforms \u2014 too many to cover in this article. So for a deeper dive, check out my video: Zapier vs Integromat (Make) \u2014 a detailed comparison.\n\nHow to get started with Zapier?\nZapier is relatively easy to use and has loads of integrations you can use out of the box.\n\nYou can tap into their ever-growing library of ready-made integrations, or you can create custom Zaps that allow you to automate more complex or niche tasks. Either way, it\u2019s a relatively simple process.\n\nAs I mentioned earlier, perhaps the best place to start is by looking at the apps that Zapier already integrates with. Chances are, you\u2019ll find tons of ready-made connections to some of your favourite tools. Personally, I rarely use these templates, but they are great for coming up with ideas, and learning what\u2019s possible.\n\nGo to the Zapier website, then navigate to Product, and then App integrations. You\u2019ll see a page like this:\n\n![Zapier](https://i.ibb.co/TB6DcnQ/1.jpg \"Zapier)\n\nClick on one of the apps, and you\u2019ll see some of the integrations Zapier offers. The below example is Gmail. Use the search box to search for another app you\u2019re using, or just click one of the ones shown to get an idea of integrations that you can set up between the two.\n\nIntegrations\n\nIf you scroll down farther, you\u2019ll see a list of automation templates for Gmail with other apps. Again, this is just a great way to generate ideas. But you can always click the \u201cTry It\u201d button to load this workflow into the Zap editor and begin playing around. Here are some examples.\n\nTemplates\n\nScroll down even farther and you\u2019ll see the triggers and actions supported by the app.\n\nTriggers and actions\n\nThis is my favourite section. If you understand the Triggers and Actions supported by the apps you use, you can start joining the dots. In the below example, just looking at the list might inspire you to add Slack messages to a Google Sheet by adding a reaction to the message\n\nTriggers and action example\n\nHow to get started with Make?\nWith Make, you can connect as many apps as you want, with as many steps as you need. Make calls these workflows \u201cscenarios.\u201d In practice, this allows you to create highly complex automations that transform all sorts of business processes.\n\nMake also offers a lot of ready-made automations, although not as many as Zapier. To explore what you can do, head to Make\u2019s website and navigate to Platform, and then Apps & services. Like the Zapier website, you\u2019ll find here all the apps you can connect together.\n\nZaper apps\n\nIf you click on an app, you can see a list of triggers, actions, and searches. You can then use and combine these to create bespoke workflows. Here are some examples for Google Sheets.\n\nGoogle Sheets examples for Make\n\nBy navigating to Solution and then Templates, you can see and search for ready-made automations that you can deploy quickly and easily. Here are some examples.\n\nMake templates\n\nIf you see one you like, simply follow the steps and give it a try."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a feature-style article showcasing how indie hackers optimize their productivity by working smarter, not harder. Gather insights from various entrepreneurs about their top productivity hacks, methodologies, and habits. Cover techniques such as time blocking, Eisenhower matrix, ICE scoring, and Pomodoro. Provide actionable takeaways for readers looking to improve their efficiency and focus."
            },
            {
                "role": "assistant",
                "content": "These indie hackers are\nworking smarter, not harder\nby James Fleischmann\n\n\"Work smarter, not harder\". Words to live by, but this can be easier said than done.\n\nHere's how productive indie hackers are working smart. \ud83d\udc47\n\nThe number one thing that keeps indie hackers productive\n\ud83d\udc64 Hiram N\u00fa\u00f1ez of Tee Tweets:\n\nA lesson hard learned: rest. When I fully disconnect and I'm not physically near my desk, I come back refreshed and feel better prepared to tackle my projects.\n\n\ud83d\udc64 Ida Radovanovic of Playground:\n\nMy \u2018why\u2019. Creating something meaningful is why I started this. So when I don\u2019t feel like working, I just go back to what my work is bringing to this world.\n\n\ud83d\udc64 Sarah Joy Calpo of Impulse Bye:\n\nSelf-awareness. Specifically, of my energy and mental state. I've learned that if I push myself to work beyond a certain point, I'm going to pay for it by being almost useless the next day. So I listen to my body and brain for when I need to take breaks or stop working, and respect that need.\n\nAlso, being self-aware of what feels motivating in the moment. I'm not someone who can stick to routines. Accepting that repetitive routines don't work for me, and finding flexible and fun approaches helps me stay productive day after day.\n\n\ud83d\udc64 Fouad Tolaib of Archie Max:\n\nAs clich\u00e9 as it may sound, it would have to be sheer determination. My determination stems from understanding the consequences of not following through and the reasons behind my actions.\n\nMotivation is fleeting and temporary; you can't rely on it. With determination, I hold myself accountable, which necessitates finding smarter ways to stay productive.\n\n\ud83d\udc64 Pieter Levels of PhotoAI:\n\nMy favorite productivity hack is saying \"no\". Source\n\n\ud83d\udc64 Tony Dinh of Typing Mind:\n\nMy best productivity hack: Work when I feel the most inspired and focused.\n\nThis has been the most significant change for me after quitting my job. Let's escape the 9-5 routine. Source\n\n\ud83d\udc64 Arvid Kahl of The Bootstrapped Founder:\n\nHere is my crazy productivity hack: Eat well and get some good sleep on a comfortable mattress. Source\n\n\ud83d\udc64 Maxime Dupr\u00e9 of Splitzy:\n\nTime block, remove all distractions \u2192 deep work.\n\nSystematize everything to remove decision-making.\n\nSleep. Well. Enough. Source\n\nBest methodologies and processes\n\ud83d\udc64 Hiram N\u00fa\u00f1ez of Tee Tweets:\n\nI use the Eisenhower matrix. To me, it's the best way of understanding what I should actually focus on. After all, if it's not important, why focus on it?\n\nThere are exceptions to this rule. When my productivity has waned, sometimes I'll do some less important things because they require less brain power.\n\n\ud83d\udc64 Sarah Joy Calpo of Impulse Bye:\n\nI use Pomodoro. However, I'm not super strict about following it. Often, I'll set a timer just to get myself to start working. If the timer ends and I'm still focused, I'll continue working until I get to a good pausing point. I try not to work for more than 90 minutes without a significant (15+ minute) break, though.\n\n\ud83d\udc64 Ida Radovanovic of Playground:\n\nFor the past two years, I have worked with the 52/17 minutes productivity rule. It\u2019s a similar idea to the Pomodoro method, but unlike the short sprints, the 52 minutes lets me focus deeply on a task.\n\nWhen I say focus, it means no extra tabs, no phone checking, just you and your task.\n\nWhen you move onto your 17-minute break, be careful. I used to spend the whole break checking my phone/socials. So I did an experiment of \u2018no screen\u2019 policy in breaks and this helped to calm my mind.\n\n\ud83d\udc64 Felix Wong of Zlides:\n\nI decide what to work on according to the ICE scoring system:\n\nI = Impact\nC = Confidence\nE = Ease\nI take a task and give it a score for each of these attributes from 1-5, then I multiply them together. The highest number is the one that I complete first.\n\n\ud83d\udc64 Hiram N\u00fa\u00f1ez of Tee Tweets:\n\nI use todo lists and organize tasks by status with a due date and priority, sorted by descending order. I focus on the status first because I try not to let too many things be in progress. This is really difficult, but I try. Otherwise, it gets even more hectic.\n\nI give the due date more importance than the priority, solely because the deadline makes sure the project keeps moving. Without a deadline, it's easy to keep putting things off (even if the priority is high).\n\nThe priority field helps sort the rest.\n\n\ud83d\udc64 Cyrus Zhang of ReadAvacado:\n\nUse the GTD method. Source\n\n\ud83d\udc64 Fouad Tolaib of Archie Max:\n\nI use todo lists and organize them by:\n\nProject\nType (writing, design, marketing, admin, research, etc.)\nOne-time or recurring\nInterval (if applicable)\nPriority\n\ud83d\udc64 Upen of Flezr:\n\nThis is what I do:\n\nBefore I close for the day, I make a list of tasks that I need to finish the next day\nStart with higher ROI tasks\nUse simple paper/pen to write down the tasks and keep striking off once done\nBookmark things that I want to read later and close them every week. Source\n\ud83d\udc64 Ida Radovanovic of Playground:\n\nI combine kanban with Trello. I am a visual person so it\u2019s really helpful to see tasks that I am focusing on and my progress.\n\n\ud83d\udc64 Sarah Joy Calpo of Impulse Bye:\n\nI use a todo list to organize the tasks that I'm working on that day or in that work session. When I'm capturing tasks to do later, I put them in the relevant project in Asana. I have an Asana Project for each of my projects, plus one for admin/miscellaneous tasks. When I make my todo list for the day, I'll grab existing tasks from these different projects by assigning them today as a due date. That way they'll show up in my week view.\n\nWithin an Asana project, I group tasks into sections: Ideas, Notes, Resources, Archive. Ideas are doable tasks, Notes are for notes on the project as a whole, Resources are links to external resources, and Archive holds completed Idea tasks. I think I prefer the Ideas/Notes/Resources grouping more than KanBan because there's less pressure to get tasks done in a linear fashion.\n\nEnjoying this roundtable? Subscribe for more how-tos, roundtables, and interviews with people in the thick of it.\n\n\nEnter email address\n\nSubscribe\nHard-earned productivity tips\n\ud83d\udca1 Tip: Eliminate distractions\n\ud83d\udc64 Hiram N\u00fa\u00f1ez of Tee Tweets:\n\nEliminating distractions is imperative. No meetings, no messages, no emails, just heads-down focus time. When you're able to get into a flow state, it's a real life hack. Not only because the quality of your work is higher, but because in those 2-3 hours of heads-down focus time, you can do what you would've otherwise done in 5-7 hours with distractions.\n\nTony Dinh of Typing Mind:\n\nUltimate productivity hack:\n\n\u2705 Code in full-screen\n\u2705 Do Not Disturb on\n\u2705 Headphones on Source\n\n\ud83d\udca1 Tip: Create phone friction\n\ud83d\udc64 Marc Louvion of Shipfast:\n\nKeep your phone in a separate room. You\u2019ll be amazed to see how a little friction makes your phone less distracting. Source\n\n\ud83d\udca1 Tip: Find purpose\n\ud83d\udc64 Hiram N\u00fa\u00f1ez of Tee Tweets:\n\nYou gotta find purpose and meaning behind your work. It has to be impactful. It doesn't have to be world-changing impactful, but it does need to have some sort of impact.\n\n\ud83d\udca1 Tip: Focus on one task\nTony Dinh of Typing Mind:\n\nI break 90% of my tasks into small chunks of work that can be done within one working session (about 3-4 hours). This includes building, testing, deploying, and writing docs.\n\nBecause of that, I can be fully focused on one task of one product on one specific day.\n\nAnd on a typical working day, I only do one task. Then, I spend the rest of the day on marketing, customer support, and interacting with the community (mostly on Twitter).\n\nOther days, I do smaller tasks with a lot of context-switching, which I find quite fun and relaxing.\n\n\ud83d\udca1 Tip: Automate (and the \"Rule of two\")\n\ud83d\udc64 Tony Dinh of Typing Mind:\n\nI try to automate everything that can possibly be automated. Especially around customer support.\n\nSelf-refund, no questions asked\nA lot of FAQs\nAdd tips and GIFs around the app UI\n\"Rule of two\": When the 2nd customer asks for something, I'll try to automate it.\n\ud83d\udca1 Tip: Track progress (not time)\n\ud83d\udc64 Fouad Tolaib of Archie Max:\n\nRather than tracking time, I focus on how much closer I am to reaching an objective. A glance at my to-do lists gives me a good indication of my progress.\n\n\ud83d\udca1 Tip: Slow down\n\ud83d\udc64 Fouad Tolaib of Archie Max:\n\nAvoid the stress of randomly releasing products just for the sake of \u201cshipping.\u201d Respect yourself and those you aim to serve with your solutions.\n\nRushing to outpace others isn't productive. And first impressions matter., particularly in today's competitive landscape enhanced by AI and no-code solutions. It's better to spend more time on R&D, producing a more appealing and well-rounded product.\n\nThe misinterpretation of the MVP concept, originally meant for expedited R&D, has done more harm than good.\n\n\ud83d\udca1 Tip: Write it down\n\ud83d\udc64 Hiram N\u00fa\u00f1ez of Tee Tweets:\n\nWrite down as much as you can, especially if you're juggling and doing a lot. I can't tell you how many times I fell into the trap of thinking \"I'll definitely remember that\", and then forgetting afterward.\n\n\ud83d\udca1 Tip: Don't become attached\n\ud83d\udc64 Fouad Tolaib of Archie Max:\n\nDon't become overly attached to achieving a result. Balance is key. Being determined without being too fixated on outcomes allows for necessary breaks to refresh your mind and sharpen your tools.\n\n\ud83d\udca1 Tip: Optimize your workspace\n\ud83d\udc64 Hiram N\u00fa\u00f1ez of Tee Tweets:\n\nA simple thing that makes me more productive is my mechanical keyboards. The audio and haptic feedback allow me to type faster and with fewer mistakes, so when I have a writing-heavy task, I'll make sure that my good keyboard is ready to go.\n\n\ud83d\udca1 Tip: Get an accountability buddy\n\ud83d\udc64 Pieter Levels of PhotoAI:\n\nFind a productivity partner, someone you mutually share your tasks with every day. This instantly makes your todo accountable. Source\n\n\ud83d\udca1 Tip: Be kind to yourself\n\ud83d\udc64 Ida Radovanovic of Playground:\n\nBe more kind to yourself. Sometimes that means resting more. Other times it means taking longer time to work on a task that you thought would be quick. And don\u2019t take it all so seriously.\n\n\ud83d\udca1 Tip: Take breaks (and naps)\n\ud83d\udc64 Rob Hope of One Page Love:\n\nI start working around 7am and finish up at 7pm every day. But I take tons of breaks to surf and run. I also nap for 18 mins every day around 3pm.\n\nWhy an 18-minute nap? Man, I\u2019ve been trying to perfect this strategy for years. So, 20+ has a longer wake time and at 30+ you are almost grumpy having to wake up. An 18-minute afternoon nap, followed by an espresso, gives me easily 2+ hours of super productive screen time.\n\n\ud83d\udca1 Tip: Clock out\n\ud83d\udc64 Rob Hope of One Page Love:\n\nI am learning to accept that side projects can never be finished.\n\nThere is unlimited optimization and marketing to do. The only true time it's done is when it's sold or the domain expires.\n\nSo most evenings I try to shut off, cook, and watch a series with my wife. I rarely work late anymore and it's awesome. I dig sleeping, ha. At the same time, my wife accepts my chosen path of building online and knows sometimes we need to focus on getting it done. No matter what.\n\n\ud83d\udca1 Tip: Dogfood your product\n\ud83d\udc64 Tony Dinh of Typing Mind:\n\nI use all of my products every day.\n\nIt's the #1 source of motivation to continue working on the products, or just to push out a new update.\n\nAlso, because of this, I often find bugs before the customers do, so that's even better.\n\n\ud83d\udca1 Tip: Hire a virtual assistant\n\ud83d\udc64 Rob Hope of One Page Love:\n\nI hired someone to help with admin 4 days a week.\n\n\ud83d\udca1 Tip: Think\n\ud83d\udc64 Arvid Kahl of The Bootstrapped Founder:\n\nAn hour of thinking is a productive hour. Source\n\n\ud83d\udca1 Tip: Plan your focus\n\ud83d\udc64 Ida Radovanovic of Playground:\n\nAt the beginning of each month, I create a \u201cMonthly focus list\u201d eg. \u201cDecember focus\u201d. It includes up to four focus areas, I have tried more, but learned that four is the optimum. Then I add measurable KPIs to each focus area. From there I break down these focus areas into weekly todo actions.\n\n\ud83d\udca1 Tip: Reflect\n\ud83d\udc64 Ida Radovanovic of Playground:\n\nReflect. I find it to be a very powerful tool to take smarter steps in the future and feel proud of yourself. This is why, at the end of each month, I stop and do a list of:\n\nWhat went well\nWhat wasn\u2019t so great\nWhat should I do next\nWe are often just on the go, but instead of execution mode, when you pause, you can learn from yourself and make better decisions.\n\n\ud83d\udca1 Tip: Create a focus ritual\n\ud83d\udc64 Sarah Joy Calpo of Impulse Bye:\n\nCreate a focus ritual. When I'm finding it really, really hard to concentrate or get started, I have a quick ritual that gets me focused:\n\nHydrate\nReaching stretches\nWarm up my hands\nTake a few deep breaths to center myself\nWrite down the first thing I want to get done\n\ud83d\udca1 Tip: Templatize\n\ud83d\udc64 Tony Dinh of Typing Mind:\n\nI keep a markdown file called \u201cSNIPPETS\u201d in each of my projects and put common reusable/project-specific code snippets in there. So when I need to create a new similar code block, it's fast. Source\n\n\ud83d\udca1 Tip: Get to work\n\ud83d\udc64 Arvid Kahl of The Bootstrapped Founder:\n\nThe truth about productivity: the more you read about it, the less productive you are \ud83d\ude09 Source\n\nSome tools to make it happen\n\ud83d\udc64 Fouad Tolaib of Archie Max:\n\nToDoist: For to-do lists and task management.\nChatGPT: For research, analysis, and proofreading.\nClose: For outreach and CRM.\nGoogle Drive: For file sharing and document management.\nEvernote: For ideation and note-taking.\n\ud83d\udc64 Hiram N\u00fa\u00f1ez of Tee Tweets:\n\nBaserow\nClickUp\n\ud83d\udc64 Rob Hope of One Page Love:\n\nAfter years of using Trello I'm now all-in on Notion. I have one team account for Love Curated shared with one other person. Then I have another team account with my wife to keep track of home loans, finances, upcoming trips, etc. I dig being organized.\n\nThen iA Writer for light lists/writing (sync'd with Dropbox). Clear app (think I bought it 8 years ago for $5) on the iPhone for shopping and temporary lists.\n\nMy desktop icons often act as a to-do list too and I'm always trying to clear those out.\n\n\ud83d\udc64 Sarah Joy Calpo of Impulse Bye:\n\nAsana: For capturing tasks, project notes, and planning my work for the day/week.\nNotebook and pen: Sometimes an idea demands to be explored on paper, not on a screen. I'll also use notebooks or loose paper for daily todos sometimes.\nTidal: Yes, the streaming platform. Music is 100% a productivity tool for me. Certain songs can help me get into flow easily, and I often use playlists or albums as Pomodoro timers.\nCalendarist: I don't consistently track my time, but when I do, I use Calendarist since it uses Google calendar events.\nHumble New Tab Page: This is a Chrome extension that displays your bookmarks on your new tab page. I have project folders with relevant links for each project that I can collapse and expand for easy access to frequently needed links (e.g. links to GitHub repos or documentation for whatever framework I'm using). Having them easily accessible means spending less time trying to remember where things are or how to get to them.\nAnd some words of encouragement\n\ud83d\udc64 Sarah Joy Calpo:\n\nRemember: Your vision is worth the work. I'm not saying it's going to be perfect and the next big thing, or even conventionally successful, but the lessons you'll learn, the skills you'll improve through doing, the growth you'll experience, all of that is going to happen for you. Keep going."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a thought-provoking post discussing the challenges of managing multiple business areas as a solo founder. Emphasize the importance of balancing product development, marketing, legal, and financial responsibilities. Invite readers to share their favorite tools and mental frameworks for managing different tracks effectively. If they have a different perspective, encourage them to share their alternative approach."
            },
            {
                "role": "assistant",
                "content": "Juggling different tracks as a solo-\nfounder: your tools/frameworks.\nby Semyon Fast\nHello!\n\nIntro on the problem:\nWorking on the product as a solo-founder, I recognized that a company success is based on multiple things done at least fine.\nThere is a building, there is a marketing, there is a legal and finance; they also divided into smaller groups. One can't be really \"bad\" at single of them because it would question the survival of the whole venture. If a product is poor, no marketing would help, and vice versa. You get the gist. That is important to take care of various things ~simultaneously.\n\nThe question: what tools or mental frameworks do you use to manage multiple tracks to keep them relevant?\n\nP.S. If you don't share this thinking -- introduce your approach, would be great to know."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write an in-depth productivity article explaining an iterative system for achieving big goals. Use a real-world example of a founder taking on a high-stakes challenge, outlining how they applied a \u2018Plan, Do, Learn\u2019 methodology. Break down the key execution strategies using acronyms (e.g., AIM, FIRE, SCAN) and provide actionable insights for readers looking to improve their execution and decision-making process."
            },
            {
                "role": "assistant",
                "content": "Aim, fire, scan: the 80/20 of executing on big projects\nA successful founder shares his productivity system. Learn the best way to make plans, execute on them, and learn from your results.\n\nBy Channing Allen\nJanuary 5, 2024\n\nThe Contract\nOn March 13, 2023, I did something crazy:\n\nI opened StickK.com, a site for making commitment contracts, and placed a bet that I would write a novel in 9 months and submit it to my literary agent by the end of the year. I told all of my friends about this contract and discussed it publicly many times, including to 100,000+ podcast listeners.\n\nThe stakes of the bet? If I failed, I would send $9,999, the maximum amount allowed by StickK, to a so-called \"anti-charity\" whose cause I didn't support.\n\nSpoiler: I succeeded with about an hour left in the year and zero gas left in the tank. I'd even found a free day to whip together an AI-based book trailer after finishing the first draft.\n\nBut here's what you need to know: even though I was confident I would pull through on the contract, there was simply no clear path to victory at the time that I signed it:\n\nI hadn't written a word of fiction in almost 10 years.\n\nI'm a good writer, but I've historically been a slow writer.\n\nI run a business which requires a lot of my time, energy, and attention.\n\nHow, then, was I so confident I would succeed?\n\nPlan, Do, Learn\nI fear not the man who has practiced 10,000 kicks once, but I fear the man who has practiced one kick 10,000 times. \u2014Bruce Lee\n\nI was confident because I've spent years mastering a simple iteration process \u2014 the plan, do, learn loop \u2014 which helps me innovate and course-correct toward just about any goal, provided I'm given a reasonable time frame.\n\nI encourage you to go read that old post if you haven't, but tl;dr:\n\nI make a plan, then do the plan, then learn from my efforts by reviewing my progress. I do this every day of the year, going on half a decade now.\n\nThese plan, do, learn loops (\"PDL loops\" for short) exist at six nested levels:\n\nQuarters\n\nMonths\n\nWeeks\n\nDays\n\nSessions: multiple per day, lasting ~3 hours each and broken up by 60-minute breaks\n\nActions: multiple per session, lasting anywhere from 5\u201360 minutes each\n\nIf it sounds simple, it is. And it isn't. Throw a kick 10,000 times and you'll find lots of nuance.\n\nMy original post hinted at my iteration speed but said little about my iteration strategy. That is, I didn't explain how I plan, how I do, or how I learn.\n\nBetter late than never. In this post, I'll talk about 20% of the factors that lead to 80% of my outcomes. They're captured in the following acronyms:\n\nAIM: Plan\n\nFIRE: Do\n\nSCAN: Learn\n\nEach of the concepts deserves its own standalone post, so I'm going to try to keep things short and sweet from here.\n\nAIM: The 80/20 of Good Planning\nAIM stands for ABZs, Interrogate, and Mandatory.\n\n1. ABZs:\nThis gem is from Shaan Puri:\n\nToo many people get caught up in the HOW. \u201cHow do I go from A to B to C to D to E etc.\u201d\n\nAll you need is the ABZ:\n\nA = where are you now\nB = next small step\nZ = end vision\n\n2. Interrogate:\nSuppose you've already made an iteration through the loop and now you're reviewing your original plan. Rather than thoughtlessly continuing down that original path, interrogate the plan and confirm that it still makes sense.\n\nAt one point while writing my novel, I reflexively started working on a new chapter because I'd included it in the original outlining process. But interrogation helped me realize that it was no longer needed, which literally saved me weeks of wasted effort.\n\nHeed Peter Drucker:\n\nAfter completing the original top-priority task, the executive resets priorities rather than moving on to number two from the original list. He asks, \u201cWhat must be done now?\u201d This generally results in new and different priorities.\n\n3. Mandatory:\nIdentify a mandatory, must-ship objective for the work iteration, and commit to shipping it regardless of the obstacles you encounter. If you can't think of a concrete objective, make it a time commitment like \"contribute 5 on-task hours to my novel this week.\"\n\nI've personally found it really effective to give myself a \"day must-ship objective\" and a \"week must-ship objective.\" A few months into my StickK contract I even started taking out $50 \"mini-contracts\" to hold myself accountable to my week must-ship objectives.\n\nFIRE: The 80/20 of Good Doing\nFIRE stands for Frontload, Improvise, Resourcefulness, and Exhaust.\n\n1. Frontload:\nMark Twain said it best:\n\nIf it's your job to eat a frog, it's best to do it first thing in the morning. And if it's your job to eat two frogs, it's best to eat the biggest one first.\n\nTo repeat what I've written elsewhere, long-term optimism for difficult projects is made out of short-term pessimism.\n\nWake up assuming everything will go wrong today. This week. This month. And execute accordingly.\n\nIf things do go wrong, you'll still be on track. If they don't, you'll be way ahead.\n\n2. Improvise:\nSpeaking of things going wrong: they will. And when they do, you'll be happy you followed the ABZ framework and didn't overplan.\n\nI use all sorts of techniques for improvising and getting unstuck. Here are just a few of them:\n\nThe BUS method: freewrite ideas, then organize them\n\ninterstitial journaling: journal as you work through tasks\n\nrubberducking: explain the problem you're trying to solve to an inanimate object\n\nNote: there's a huge difference between improvising on the iteration path and improvising on the iteration itself. Using the ABZ framework, a legitimate improvisation might be rethinking how to achieve B, but it will almost never be rethinking B itself. As a rule with few exceptions, you want to move forward through iteration loops, not backward. (Too often, moving backward is a sign of problems with doing, not problems with planning.)\n\n3. Resourcefulness:\nHuman brains suck at most things. Instead of relying only on mental resources like willpower and memory, use extra-mental resources for added leverage.\n\nThis is a book-length topic and perhaps the main secret of my success, so forgive me the vagueness of simply bucketing extra-mental resources into two categories and leaving it at that for now: 1) The Three Me's and 2) The Two We's:\n\nThe Three Me's:\n\nTechniques, e.g. the BUS method\n\nTools, e.g. standard operating procedures, ChatGPT\n\nTerritories, i.e. environmental constraints on your behavior, e.g. putting your phone on DND mode\n\nThe Two We's:\n\nTeachers, e.g. books, mentors, content creators\n\nTeams, e.g. personal assistants\n\n4. Exhaust:\nDon't take the \"exhaust\" term too literally here. Burnout is real, and I'm not suggesting you should do that. Instead I mean that if you decide to do an hour of work, try to work a 60-minute hour rather than, say, a 30-minute hour filled with continuous partial attention.\n\nEasy to do when you're passionate or things are going well. Less easy when you're stuck or bored, and discipline is required.\n\nEnforced time constraints work wonders here. My own convention is to do intensive work sessions for set blocks of time then self-impose 30- to 60-minute cooldown periods where I relax and recharge. I use a website blocker called Freedom for this. I did a deep-dive on this process here.\n\nSCAN: The 80/20 of Good Learning\nSCAN stands for Score, Critique, Adjust, and Notes.\n\n1. Score:\nScoring helps you measure your performance objectively. If you don't do it, you end up lying to yourself without knowing you're lying to yourself.\n\nAgain, heed Drucker:\n\nOne company chairman was absolutely certain that he divided his time roughly into three parts. One third he thought he was spending with his senior men. One third he thought he spent with his important customers. And one third he thought was devoted to community activities.\n\nThe actual record of his activities over six weeks brought out clearly that he spent almost no time in any of these areas. These were the tasks on which he knew he should spend time \u2014 and therefore memory, obliging as usual, told him that these were the tasks on which he actually had spent his time.\n\nI like to score (or at least track) two things: inputs and outputs.\n\nBy inputs, I really just mean time: \"How many hours did I put in versus the amount of hours I was supposed to put in?\" Outputs is more self-explanatory: \"Did I ship my must-ship objective?\"\n\nBoth can be scored with a simple 0 or 1. (My personal scoring systems have become a lot more sophisticated after years of iteration, but I started with something like this.)\n\n2. Critique:\nIf you take the time to look, you will always, always, always identify at least one flaw in your performance that you can improve upon. But if you don't look, that flaw will often go unnoticed for many iterations, or even forever.\n\nWhat I do is identify at least one of these flaws at the end of each iteration cycle.\n\n3. Adjust:\nIt's not enough to notice a flaw. You also want to take action. And in my experience, simply saying, \"Okay, next time I won't do [flaw]\" is bullshit 90% of the time.\n\nOne of my bad habits early on in the process of working on the novel was putting other work in front of it: answering work emails, chatting with my team, doing morning routine tasks, etc. By the time I got around to writing \u2014 which requires an enormous amount of mental energy \u2014 I was often drained.\n\nSo I made an adjustment: within 30 minutes of waking up, I needed to get started on the manuscript and knock out at least 2 hours of work before opening the pandora's box of digital distractions.\n\n4. Notes:\nWrite down how things went during this iteration, and why. Benjamin Franklin began each day by asking himself in his journal: \u201cWhat good shall I do this day?\u201d Then, at the end of the day, he'd ask \u201cWhat good have I done today?\u201d\n\nIf it's good enough for Big Ben, it's good enough for you. The first reason why documenting your progress is good is because it leaves a record for you to investigate later. You'll be less likely to make the same mistakes over again without being able to lie to yourself about it.\n\nThe second reason why this is good is because writing is thinking. Often, the act of writing your learnings from the iteration will lead to far greater clarity about your performance than you could gain otherwise. It's similar to the process of rubberducking."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Create a first-person narrative post from an entrepreneur who manages multiple startups. Explain their approach to time management, prioritization, and delegation. Introduce a structured framework\u2014such as a 'Maintenance vs. Expansion' model\u2014that helps them balance ongoing commitments with growth opportunities. Discuss the trade-offs, challenges, and benefits of handling multiple ventures simultaneously."
            },
            {
                "role": "assistant",
                "content": "How I juggle 8 startups\nat the same time\nby Arian Adeli\n\nThe headline is a bit of an exaggeration. I'm not Elon Musk. But I am bootstrapping a venture studio, called Evernomic, with a focus on the media industry.\n\nMy Projects\nSome of our projects include:\n\nNewsletters like Internet Is Beautiful and Discovery Dose\nSaaS projects like Feedboard\nA local newspaper in the Netherlands\nA private deal-sourcing service\nWebsites like Find Newsletters\nMind you, Evernomic itself takes a lot of work. We also have contracts with other companies and many side projects, including:\n\nQuora spaces with 2M+ monthly views\nTelegram Channels with 20,000+ subscribers\nAn exclusive community of writers at prestigious publications\nAn internal API for detecting the \"interestingness\" of headlines\nThis is normally a liability for me in business interactions but I am still in university as well. People often look down on that and underestimate me for it but it's something I aim to go through regardless of my success in business.\n\nPlus, I hate nothing more than being one-dimensional. I don't just want to be good at business or any ONE other field. I workout and take care of my health, I spend much of my time learning new things, I like to socialize and make memories, I pay attention to how I look, and I pursue hobbies that push me forward like writing and public speaking.\n\nMy Process\nNow, this begs the question of how I manage my time to not fall behind on my responsibilities.\n\nI'd like to start by saying I'm a strong believer of continuous improvement. I'm constantly looking at ways to become more efficient and cut down unnecessary responsibilities. I'm by no means perfect at time management.\n\nBeyond that, I have developed what I call a Maintenance/Expansion Framework.\n\nI divide my workload and things that require headspace into two categories:\n\nMaintenance: ongoing commitments that require my continued attention on a regular basis that are necessary for me to keep going in the same trajectory. Essentially, things I need to do to maintain my current progress.\n\nExpansion: one-time or open-ended tasks that improve the overall condition of my life whether it's in business, fitness or happiness.\n\nThis may sound controversial to openly admit but I avoid commitments, a.k.a Maintenance Tasks, at all costs. That's the danger zone. It takes away from my ability to expand and take on more things when I'm faced with better opportunities in the future.\n\nI like to always leave some free margin to try new things, waste some time, let things go wrong without detrimental consequences and so on. Therefore, I'm very picky with things I commit to and I try to minimize them.\n\nConversely, I'm unafraid of the Expansion side of things. I can turn months into hell, taking on several things simultaneously, as long as I know it will be over soon. The only decisions that truly concern me are those that lock me into a certain future.\n\nA typical agenda for me looks something like this:\n\nMy daily routine\nMaintenance Tasks\n3-4 Expansions Tasks at a time\nThis division gives a game-changing order to my life. I know what the bare minimum of what I should be doing is and it lets me focus on multiple domains within my life and business simultaneously.\n\nWithin Evernomic, I try to build systems and frameworks through which we push our projects which gives us a streamlined, efficient approach. Plus, things get delegated effectively which minimizes commitments to low-yielding tasks on the higher levels.\n\nNaturally, focusing on so many things simultaneously has its drawbacks too. I often fail to match deadlines, I often don't end up acting on plans I've had, I often don't reach my goals. In fact, at any given moment, I'm probably not doing too well.\n\nHowever, I've learned that time has a magic of its own. Whenever I've looked back on my progress over a few months, I was pleasantly surprised. Now, I've decided to trust the process and keep pushing forward regardless.\n\nHow do you deal with time management? Any feedback on how to improve my approach?\n\nFollow me @arianadeliii for more insights from my journey!"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a counterintuitive productivity post exploring why doing nothing can actually enhance focus. Discuss common struggles with distractions and provide a practical method that forces the brain to choose between working or doing nothing at all. Use real-life experimentation with the Pomodoro Technique to illustrate the impact of this approach."
            },
            {
                "role": "assistant",
                "content": "The best way to focus? Do nothing.\nby Jacob Robinson\n\nThis is a quick blog post to talk about an important thing I recently learned when it comes to focusing on a task at hand.\n\nFocusing is hard\nOf course, I\u2019ve been trying to improve my productivity more now that I am self-employed and have a higher need to focus on work. I\u2019ve been trying to figure out the best path to productivity, specifically the best ways to focus on a single task over an extended (1+ hour) period of time.\n\nIn the past, I\u2019ve had trouble with this across the board. I often switch between tasks, which to be fair can help reduce burnout if the tasks are diversified and separate from one another. However, if I have too much spread and keep switching without being able to focus, I\u2019ll never finish anything.\n\nA counterintuitive approach\nSo, I\u2019ve been experimenting with different methods to find the best way to focus. I think I figured it out, at least for me: and it\u2019s to do nothing.\n\nNow, this sounds weird, but let me add more context. I use a Pomodoro timer to manage my productivity, which is very common and helpful. However, I still struggle sometimes to sit down and work the entire time for the Pomodoro. Sometimes I\u2019ll get distracted and forget all about the Pomodoro timer, and when I come back, it\u2019s already been five Pomodoros past, and I haven\u2019t done any work for the entire day.\n\nWhat I\u2019ve decided to do is give myself two choices. I can either work during a Pomodoro or do nothing. I can walk around, think, stand, or sit, but I can\u2019t go on Discord, Reddit, YouTube, or any other distractions. That way, if I\u2019m burned out from a task, I can get up from my chair and start walking around. Sometimes, I go through an entire Pomodoro timer doing nothing but walking around my room.\n\nMy reasoning\nWhy is this better than getting distracted? First, when you\u2019re doing nothing, there\u2019s no time dilation, unlike when you\u2019re going on Reddit or YouTube. This means that while I\u2019m liking to miss one Pomodoro, I\u2019m very unlikely to miss five of them.\n\nSecond, doing nothing is very boring, even more boring than working. So, sometimes, I just find that if I\u2019m left with these two choices, I\u2019ll go back and be like, \u201cOkay, I\u2019m done walking around. I thought about what I need to think about. Let me just go back to this work.\u201d\n\nThat\u2019s how I\u2019ve figured out the best focus for my Pomodoro timer. It\u2019s definitely something that I think would be valuable for other people to try, too. I still haven\u2019t perfected this formula, but I\u2019m still working through it, and of course, there will be a blog post when I figure out the perfect distractionless work environment (if it even exists). But that\u2019s where it stands right now, and I feel like that\u2019s a strategy that\u2019s worked really well for me."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a reflective blog post about a week where productivity expectations didn\u2019t match reality. Break down five key lessons learned, such as the importance of focusing on impact, managing large tasks, celebrating small wins, removing distractions, and recognizing long-term progress. End with a motivational note encouraging readers to embrace unproductive weeks as part of the process."
            },
            {
                "role": "assistant",
                "content": "Expectation vs reality. What I learned\nfrom a non-productive week.\nby Wyatt Feaster\n\nI think everyone has weeks where they feel like they worked on many things, but, in reality, they feel like they didn\u2019t complete much. Well, this past week was one of those for me.\n\nI am working on accepting that weeks like that happen and move forward, but also learn to look back and reflect on why it happened and learn from it.\n\nHere are 5 things I took away from an unproductive week\u2026\n\n1. Focus on impact\n3 large tasks are too many to focus on daily, with only about 3 hours of availability.\n\nI need to reduce the amount of things to focus on. I need to focus on one KEY thing each day. This thing should be the MOST impactful thing.\n\n2. Accept that large tasks take time\nTasks bleed over because they are large or require more in-depth thinking.\n\nThat is okay. Accept that not every task I put on this list will be accomplished. Don\u2019t beat yourself up because you didn\u2019t get the entire task complete.\n\n3. Celebrate the wins\nEven if it wasn\u2019t my most productive week, you still got things done. Something is better than nothing.\n\n4. Remove distractions\nOn weeks I have trouble focusing, I really need to remove distractions.\n\nI need to mute notifications, get off my phone, go to a different environment, and talk with someone about my project to get my brain heading in the right direction.\n\n5. My worst is still better than my best\nThroughout my journey of building emailemu.com I implemented a routine that has helped me become more productive.\n\nWhat I learned from that is that even on my worst weeks or days of productivity, they are still better than my BEST days in previous years. That is progress to celebrate.\n\nLog of my week\nFor those curious, here is the actual list of things I set to accomplish each day, vs what I got done.\n\nMonday\n\u2713 Write Blog Post\nAdd Weekly Brands to Emailemu.com\n\u2713 Meet with Devs\nTuesday\n\u2713Add Brands to Site\nDraft Newsletter Email\nFinishing Pricing Page for website\nWednesday\nAdd newsletters signup to the site\nAdd pricing page\n\u2713 Write and send Email\nThursday\nComplete pricing page\n\u2713 Write blog and post it\n\u2713 Add newsletter signup sections\nFriday\nSchedule social posts\n\u2713 Prep design for calls with marketers\n\u2713 Create interview questions for calls\nSaturday\n\u2713 Schedule social posts\nComplete Pricing page\nBlog\nSunday\nBlog\nPricing page\nComplete Pricing Page\nIn summary, know it will happen, accept it, and move on. Don\u2019t let it define you or make you give up. Keep moving forward.\n\nWhat about you?\nWhat about y\u2019all? What do you do about unproductive weeks?\n\nWhat have you learned?"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a practical guide on staying consistent while building an indie startup. Introduce three powerful techniques\u2014mindset shift (perspective), accountability groups (mastermind), and direct peer accountability (one-on-one partnerships). Use real-life examples to illustrate how each method improves long-term commitment and execution."
            },
            {
                "role": "assistant",
                "content": "3 tricks to be more\nconsistent with your project\nby Ludwig\n\nConsistency is an asshole.\n\nBut it\u2019s the essential factor that separates success from failure in a challenging project like building an indie startup.\n\nHow did we learn from Atomic Habits: everyone shares the same goals \u2014 to achieve success in some way \u2014 but since not all do, it must be more the execution that matters.\n\nThere have been three tricks, or tools, that tremendously helped me to stay consistent (=execution) with my project of developing the automatic video editor flowtapes.com, starting a YouTube channel (with 4 videos so far lol but hey), and especially doing the required but unlovely work of selling (doesn\u2019t everyone just want to hang about in their basement and keep typing clever instructions into VSCode).\n\nThe tools are ordered by the impact they had on my work, starting with third place.\n\nThe perspective\nThe third one, perspective, is about how you look at the work you do. It\u2019s about mindfulness and focusing on the process instead of the results (hashtag enjoying the journey).\n\nThis is the main weapon against two nasty modes of mental dissonance we often unintentionally slip into \u2014 1) comparing our work to other people\u2019s work, and 2) comparing our current state to an envisioned future state.\n\nBoth are bad, they drag us away from the here and now and leave us feeling it\u2019s all not enough, we\u2019re taking too long, we\u2019re doing something wrong. But it\u2019s just not true. Think of what you achieved today, yesterday, last week, and of the entire progress you\u2019ve made since you started.\n\nAnd that is where mindfulness, so intentionally focusing on the work we want to achieve today, helps us get the right perspective. It helps us diminish the feeling of dissonance (me <> others, me today <> me in the future). Remember:\n\nWe are always comparing our backend to other people\u2019s frontend, because that\u2019s the only side we can see (and they want us to see).\nThe only moment in which we can make stuff happen is now. The future is expectation, the past is identity. Both are illusions, we can only make progress in the present.\nThis mental attitude makes me not lose the fun while working through the long, slow grind of building a business. The often 1-2 years of putting a lot into it, while getting very little out. Focusing on the input and having fun making progress step by step made me work quite effortlessly through weekend after weekend and made me even use most of my vacation days (when I was still on the day job) to work on my indie startup.\n\nBut our project can obviously not always be fun. I love being in developer\u2019s tunnel for 10 hours straight but that doesn\u2019t sell my product, I have to go out there and act outside my comfort zone.\n\nSo fun and mindfulness can not be the only tools for rock-solid consistency here.\n\nThe mastermind\nThe second key tool for me was to tackle my work not solo but to interact with like-minded people who're in a similar situation as me.\n\nThis can mean to take part in discussions on Reddit, or here on indiehackers, or to chat in the comments of a Twitter post.\n\nIt makes you realize there are others dealing with the same challenges like you do. You can suddenly share your experiences with people who really understand.\n\nBut instead of interacting with a large base of people in public forums, which is often impersonal, it\u2019s even more powerful to join forces with a small circle, like with a so-called mastermind group. These consist of usually 5 to 10 members who hold regular meetings, like a monthly in-person one, or a weekly video call in which they discuss progress, roadblocks, goals etc.\n\nThe advantage of such a group compared to participating in a discussion on Reddit is it\u2019s way more personal. You get to know your fellows and their projects as they do yours, which creates a much stronger accountability energy than talking to strangers on the internet.\n\nAfter around 2 years of indie hacking on my own, I joined forces with such a group and the experience has been incredibly supportive. Everyone is working on their own software product and we talk about issues like how to improve one\u2019s pricing or how to fix SEO performance and we also share our weekly goals. Which creates another nudge to meet your goals as others are now asking about them. And that group I had just found here on indiehackers in the comments.\n\nBut we can take the not-work-alone route even one step further.\n\nThe girlfriend\nIn the same way in which a personal group has stronger accountability energy than the public internet, just one direct accountability partner has been unbeatable for me.\n\nBecause with one accountability partner you can\u2019t keep a low profile. They will come, they will want to know if you completed your marketing to-dos yet, and they will keep bugging you until you have.\n\nIn my case my accountability partner is my girlfriend but in general it can be anyone. It just needs to be someone who is capable of riding your ass. And in case you wonder, yep my girlfriend is. I just think it\u2019s beneficial when that person operates similarly like you do. For example by having an entrepreneurial side as well, so they can truly relate to your work and goals.\n\nSome people also pay for an accountability partner and a weekly meeting with them, which i can imagine increases the stakes even more. But why not find someone here on indiehackers or through a mastermind group.\n\nWith my \u201caccountability partner\u201d I do a weekly goal session each week on Monday, in which we go over last week\u2019s goals, discuss progress, challenges, roadblocks and so on, and set new goals for the coming week. And ride asses. These goddamn weekly sessions have caused me high blood pressure many times but they also made me accomplish what I would otherwise still be procrastinating on today..\n\nThe end\nIt turns out, having built the mental strength to focus on the work in front of me (the input) helped tremendously to stay consistent in the grind.\n\nBut other people holding me accountable unleashed a level of consistency I couldn't tap into while toiling away solo in my metaphorical basement.\n\nAnd while it comes natural, particularly for us technical solo founders, to stay in our comfy tunnel until the end of times, stepping out of it and connecting with others over our pursuits is incredibly more rewarding."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write an inspiring blog post about the power of taking action to gain clarity. Explain how overthinking leads to paralysis and introduce four key principles: Learning by Doing, Just-In-Time Knowledge, Reflection, and Embracing the Unknown. Use references to psychological studies and well-known thinkers to reinforce the message."
            },
            {
                "role": "assistant",
                "content": "Action Breeds Clarity\nby Martin Baun\n\nTaking action has been a life-changing decision for me. It has brought a lot of clarity to my life and day-to-day activities. I get a lot of tasks done and improve myself consistently as well. It has put me on the path to slaying complacency, eliminating procrastination, and fulfilling my immense potential. These are four principles of how action breeds clarity, and how I implement them.\n\nimigur.com\n\n1. Learning by Doing\nLearning by doing stops us from overthinking things. We are only human. We will never be sure of every decision we make. We have an innate tendency to overthink and analyze situations before taking action.\n\nOverthinking leads to what is commonly known as \"analysis paralysis.\" It blurs our perspective, creates confusion, and stagnates our progress. Overthinking creates multiple outcomes that lead to increased anxiety, dissatisfaction, and even paralysis. This concept; was theorized by psychologist Barry Schwartz in his book \u201cThe Paradox of Choice: Why More Is Less.\u201d\n\nSchwartz argues that having choices is generally considered a positive aspect of modern life. The abundance of options in various domains\u2014such as career paths and life decisions\u2014can overwhelm individuals and hinder their ability to make effective decisions. We should use our time efficiently and learn in the process. Do not overthink things.\n\n2. Just-In-Time Knowledge Over Just-In-Case Knowledge\n\u201cJust-in-time knowledge\u201d refers to acquiring skills and information when they are immediately needed for a specific task, problem, or situation. \u201cJust-in-case knowledge\u201d refers to learning skills and information in preparation for future needs.\n\nBoth approaches have their advantages. Why is this knowledge important? A lot of people read, watch videos, and consume information that they are unlikely to put into practice. This knowledge is more of an entertainment source than it is a learned skill.\n\nWe are more likely to change a light bulb than apply survival tactics when faced with a lion. There\u2019s nothing wrong with knowing how to survive a lion attack but it\u2019s very unlikely that you\u2019ll use that knowledge. People who prioritize this type of knowledge are also prone to procrastination. They focus solely on what could happen. This keeps them in an endless cycle of worry and anxiety that prevents them from accomplishing the goals they\u2019ve set for themselves.\n\nWe prioritize just-in-time knowledge and reference Tim Ferris, who said, \u201cPrefer just-in-time knowledge over just-in-case knowledge.\u201d\n\n3. Do and Reflect\nGetting things done while learning relevant aspects is good for our growth. Reflection gives us new perspectives of ourselves that allow us to analyze and improve on areas we are weak.\n\nIt is important to see your goals through. Doing so staves off doubt, depression, and anxiety. It builds your character in the process, putting you in a position to continually improve. Resist the temptation to quit or give up. Do not second-guess yourself by wondering if you should do something else instead. Guard your mind against self-doubt and stay focused on the task at hand.\n\nTake action over your patterns, routines, and nuances. Don\u2019t allow complacency to creep in, as it can drain you of all your potential, leaving you depressed and unfulfilled. Adapt to changing circumstances and build on routines that make you better.\n\n4. Building Confidence & Embracing the Unknown\nThe action involves stepping into the unknown. Embracing uncertainty allows us to confront fears and doubts that hinder progress. Navigating through uncharted territories is scary and unpleasant. However, doing so builds and grows confidence. This confidence stems from realizing we can overcome challenges and learn from them.\n\nOverthinking can create worst-case scenarios that fill us with fear. Fear can hold you back from achieving your potential. Take the leap of faith into the unknown, and overcome the challenges that await you. As David Goggins says, \u201cEmbrace the suck!\u201d\n\nIt might not be pleasurable in the beginning, but you will learn how to deal with it. You\u2019ll master how to navigate uncharted waters and come out the other side stronger than before.\n\nStop Overthinking\nAction breeds clarity and encapsulates a fundamental truth that transcends various aspects of life. It allows us to break free from the constraints of overthinking. We gain insights, experiences, and wisdom that lead to a clearer understanding of ourselves and our environment.\n\nThe courage to take action propels us toward our goals while fostering adaptability and self-confidence. Action holds the key to the clarity we seek.\n\nThe world is your oyster. Stop overthinking and act starting today.\n\nFor these and more thoughts, guides and insights visit my blog at martinbaun.com"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a thought-provoking essay challenging traditional goal-setting. Argue that focusing on continuous progress and alignment with personal values is more effective than rigid goals. Use personal experience and quotes from successful entrepreneurs to support the perspective. Encourage readers to rethink their approach to productivity."
            },
            {
                "role": "assistant",
                "content": "Why I'm happier and more\nproductive by not setting goals.\nby Mac Martine - The SaaS Bootstrapper\n\nThe most important measurement for life improvement is not in goal-tracking, but in ensuring we progress towards who we want to be and how we spend our time.\n\nThe reason that most of us are unhappy most of the time is that we set our goals not for the person we\u2019re going to be when we reach them, but we set our goals for the person we are when we set them. -Jim Coudal, Founder of Field Notes\n\nPeople often stress the importance of creating goals with measurable outcomes. While this may be useful in some cases such as building a house on schedule, we should be careful applying goals to everything we do.\n\nGoals are often arbitrary.\n\nGoals come and go.\n\nWe can get bogged down setting goals that we don't sustain because they start to feel like we're trying to check boxes. Or our ideals change before we reach the goal.\n\nMore importantly, the goals we set often have an underlying motivation; a direction we want to move towards, to which goals can make us lose track of our underlying intention.\n\nIt's not going to the gym three days a week that we want - we want to feel and look good.\n\nWe can hit our goal and still look and feel like crap.\n\nWe can also progress towards our objective by incorporating exercise into our day-to-day and eating well, and there's no specific time when it warrants checking a box and moving on.\n\nWhen we look at what we truly want, and what is most important to us, we find not a destination we want to reach, but a path we want to be on.\n\nHappiness, peace, strong relationships, a sense of belonging, a sense of freedom, even wealth - these are endless pursuits that have no bounds.\n\nThey are only measurable in the context of our past - for when we look back over any given period, it will be clear whether or not we've made progress, and that's ultimately all that's important. They are not goals we can or should check off and move on from.\n\nWhen you shift from 1st to 2nd, 1st is behind you. Then from 2nd to 3rd, 2nd is behind you.\nI approach things continuously, not in stops. I just want to keep going\u200a\u2014\u200awhatever happens along the way is just what happens\n-Jason Fried, CEO of 37signals\n\nLife contains a lot of variables and everything is ever-evolving.\n\nWe should get clear on the general direction we want our life to go in, then frequently assess what steps we can take now to move in that direction.\n\nI don't make New Year's resolutions or pre-defined goals.\n\nThe direction I want my life to go hasn't changed much in many years:\n\nRemain healthy\nSpend time with my wife and kids\nCreate and maintain strong relationships\nHave control over how I spend my time\nDo work I enjoy and am proud of\nSometimes certain things are weighted more heavily than others or require more time. I may cycle through projects, and acquire or lose (hopefully not) a friend, but where I want my momentum to go is relatively static.\n\nYet daily I review my options for what will move the needle the most to maintain the right momentum.\n\nIf you enjoyed this, subscribe to the SaaS bootstrapper series where I share the behind-the-scenes of my past 7-figure bootstrapper journey, and what I'm doing now:"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a deep-dive post on how history\u2019s greatest minds\u2014Da Vinci, Einstein, Curie, Jobs, Feynman\u2014developed their intelligence and creativity. Break down their key thinking techniques and provide actionable steps for readers to train their brains. Include sections on brain plasticity, creative habits, memory techniques, and learning strategies."
            },
            {
                "role": "assistant",
                "content": "I Studied The Smartest Minds In\nHistory So You Can Crush it In 2024\nby Zaur T\n\nHey hackers, it's Zaur from 404trends.com I'm back with another article to make your life better.\n\nThe first thing I want to tell you is - You\u2019re a creative genius. If you're thinking otherwise, let me prove it.\n\nI know it's long read, you can bookmark it to read later. Or as I usually do, reading in chunks for better understanding.\n\nSo hear me out.\n\nYou\u2019re not stuck with the brain you were born with.\nWe often look at people like Leonardo da Vinci, Albert Einstein, Marie Curie, Steve jobs or Richard Feynman as if they were born with superhuman abilities.\n\nBut the reality is, they weren\u2019t so different from you and me. The key difference lies in how they trained their brains to think differently.\n\nDa Vinci was known for his insatiable curiosity. He loved connecting ideas from different fields. He was a keen observer, always watching the world around him. Recorded his thoughts and drew sketches in his notebooks.\n\nEinstein had a unique way of thinking. He would imagine hard-to-grasp ideas in more relatable ways. He called these his thought experiments.\n\nCurie was known for her strong commitment to her work. She was very careful and organized in how she did her research. Interesting, right? Let me tell you something else.\n\nMarie Curie/Lyceum \n\nYour brain is a fascinating mechanism.\n\nThink of your brain like a toolbox.\n\nInside are different tools. They help you think. They help you remember. They help you pay attention. It has billions of tiny parts called neurons. They talk to each other using electricity and chemicals. This makes patterns called brain waves. Different patterns mean your brain is in different states, like being focused or relaxed.\n\nBrain plasticity is also called neuroplasticity. It is the base of human experiences. Everything you do starts in the brain.\n\nAll physical, mental, and emotional activities of human beings start from and revolve around the brain. All of these are possible because of neuroplasticity.\n\nThe brain can rewire itself based on new experiences. Like a muscle, the more you exercise certain parts of your brain, the stronger those neural connections become. As a result, you\u2019ll sharpen your memory and logical thinking and become healthier and happier.\n\nLet go of the notion that your brain is fixed and unchangeable.\n\nYou have the power to shape your mind and unlock your creative potential. Let\u2019s have a look at the methods to help you develop these new cells and develop brain plasticity.\n\nUnlock Your Natural Creativity\nThink of something you\u2019ve always wanted to try.\n\nIf it\u2019s learning new skills. Be precise about what you aim to achieve. For example, learn presentational skills. Or study a new programming language.\n\nExpand your professional network. Meeting people from different backgrounds and experiences can expose you to new ideas, perspectives, and knowledge. This cognitive stimulation can help keep your brain active and engaged.\n\nChange your environment. Steve Jobs is a good example. He left college. He went to India and saw different ways of life. In 1974, Steve Jobs and his friend Dan Kottke went to India. They wanted to find spiritual meaning. They wanted to understand life better. He started practicing Zen Buddhism. Which taught him to focus on simplicity, intuition, and the present moment. In 1976, Jobs co-founded Apple with Steve Wozniak. His trip to India changed his life. It changed how he did business and how he made products. He focused on products that were not only look good but made people feel something.\n\nSteve Jobs / Tom Munnecke Getty Images\n\nWhen you travel to a new country, you\u2019re forced to navigate unfamiliar customs, languages, and ways of life. This mental exercise strengthens your brain\u2019s ability to adapt and think creatively. For instance, trying to communicate with locals in a foreign language activates different neural pathways than your native tongue.\n\nTeach Someone\nTeaching is essential for learning. Teaching is crucial not only for students but also for the teacher. It forces him to review and reassess his knowledge, keep his mind active, and occasionally lead to new ideas and approaches.\n\nIf you\u2019re thinking that you don\u2019t know what to teach, well, you do. Everyone has something they know more than others. Think about what other people are asking for advice from you. Or try to remember something that you learned recently.\n\nOne of the fastest ways to start teaching is to start writing online. Just pick up the topic you\u2019re the most excited about. Learn the hell out of it. And write in your own words about this topic. It\u2019s one of the best ways to learn and to teach (and grow your audience as a bonus).\n\nRichard Feynman / Mojologic blog\n\nRichard Feynman, a famous physicist who won a Nobel Prize, had a great way of learning and explaining things. He believed that when something seems too complicated or uses a lot of fancy words, it\u2019s often because someone doesn\u2019t really understand it well. His method for learning has four simple steps, and the one of them was explain it like you\u2019re teaching a child. Explaining a concept like you\u2019re teaching it to a child is effective because it helps you break down complex ideas into simpler terms. This makes you understand the topic better and clarify any confusing parts.\n\nCreate Your Memory Palace\nThe method of loci, also known as the memory palace technique, is a mnemonic exercise that involves visualizing familiar locations to help remember information.\n\nHere\u2019s how it works.\n\nChoose a familiar location or route as your \u201cmemory palace\u201d. This could be your home, a frequently walked path, or a well-known building. For example, imagine your house is like a personal memory storage unit. Each room represents a different section to store information.\n\nIdentify specific locations or \u201cloci\u201d within your memory palace. These serve as the storage points for the information you want to remember. For example, in your home, the loci could be your front door, couch, kitchen sink, dining table, etc. The locations should be distinct and follow a logical order.\n\nNow, associate each piece of information with vivid, bizarre, or exaggerated mental images.\n\nThe more visually striking and memorable the image, the better. Let\u2019s say you need to remember a grocery list: eggs, milk, bread, apples, and cheese. In the living room, picture a gigantic egg cracked open right in the middle of the floor. A weird and silly image like that will stick in your mind. Go to the kitchen and see a huge puddle of milk all over the counter and floor. Then, the bathroom door is blocked by an enormous loaf of bread. And so on.\n\nTo recall the list later, just mentally revisit each room of your home in order. The bizarre images you planted will pop back into your mind \u2014 the egg for eggs, milk puddle for milk, and so on.\n\nMemory Palace / artofmemory\n\nWith practice, you can create multiple memory palaces for different topics or categories of information. You can use this technique for any type of information \u2014 not just lists (like storing your crypto wallet key). The rooms of your home become your personalized memory palaces to store data in an organized, visual way. It may seem strange at first, but giving your brain memorable visual hooks is a powerful trick for remembering almost anything.\n\nFictional character, Sherlock Holmes, the famous detective created by Arthur Conan Doyle is often portrayed as using a \u201cmemory palace\u201d (a variation of the method of loci) to store and recall vast amounts of information\n\nAbsorb Everything, Like a Kid In a Candy Store\nReading gives your brain a workout.\n\nIt strengthens the brain\u2019s neural networks and increases connectivity in areas related to language, sensations, and movement. Reading also improves your concentration and enhances your imagination and creativity.\n\nDon\u2019t limit yourself to books within your area of expertise. By reading across different domains, you give your brain a rich variety of knowledge to work with. This allows you to connect ideas from separate fields in novel ways.\n\nExplore beyond bestsellers. When you read the same popular books as everyone else, you gain knowledge, but it\u2019s the same knowledge that many others have. Best-selling books are often written to appeal to a broad audience, so they may not dive as deep into specialized topics.\n\nActively seek out lesser-known works in fields that interest you. The diverse knowledge you absorb can lead to unique insights that set you apart from the crowd.\n\nLeonardo Da Vinci / Everett Collection Shutterstock\n\nLeonardo da Vinci had a remarkable ability to make connections across different disciplines. He integrated knowledge from areas such as art, science, anatomy, and engineering, allowing him to approach problems from diverse perspectives and create innovative solutions. This interdisciplinary approach was a key factor in his genius and groundbreaking work.\n\nOriginally posted here"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a productivity guide advocating for weekly planning instead of long-term goal-setting. Explain why planning for shorter timeframes leads to greater adaptability and better energy management. Provide a structured framework for identifying high-energy, medium-energy, and low-energy tasks, along with a method for choosing the most important weekly goals."
            },
            {
                "role": "assistant",
                "content": "Plan your week, not your whole life\nby Akshay Raveendran\n\nI have been struggling with using my time most effectively and efficiently so that my productivity is high and simultaneously I don\u2019t feel burn out.\n\nI tried a lot of different methods, but nothing worked perfectly for me. Then, I accidentally discovered this method while I was scheduling my goals.\n\nMy goal was to increase my productivity without sacrificing the quality of my work and life!\n\nI wanted to peacefully work on tasks at hand without worrying about millions of other things. At the end of the day, I wanted to feel satisfied and fulfilled. You know, I am a creative and highly energetic person.\n\nI get thousands of ideas and want to learn millions of things. So no matter how hard I work, no matter how intelligently I develop something, I always felt like something is still missing. A feeling of guilt was always lingering in my heart.\n\nOne fine day it occurred to me that I am not a superman! I have a limited stock of energy which diminishes with the Sun going down. Besides that, there\u2019s a constant assault on my resourcefulness by various factors outside of my control.\n\nIn short, though I have 24 hours a day, I have much fewer hours in which I can be highly productive.\n\nI have already been aware of and applying the well-established time management techniques like the 80/20 rule (Pareto\u2019s principle), urgent vs. important, prioritizing your tasks, starting with the most challenging things first, etc. But something was missing.\n\nSomehow nothing was giving me peace of mind. With my headphones on, listening to focus@will tunes, shunned from external noises, I was deeply absorbed in thoughts, trying to understand why it\u2019s not working for me. Finally, a little voice in my head told me to get up and get to actual work; you are wasting time.\n\nAnd then, I realized I had set unrealistic expectations for myself. No matter how ambitious I am, no matter how talented I am, there\u2019s a limit to how much I can \u201cproduce\u201d in a given time.\n\nAnd I also realized that even if I get 24 hours to work, I am restricted by the amount of energy I have. And then there was a Eureka moment!\n\nAll the pieces of the puzzle fall in place. I immediately recorded my thoughts, and here is the framework that you can use to achieve your goals peacefully!\n\nHere\u2019s the first principle: Long-term planning is guesswork!\n\nImage description\n\nThe future can\u2019t be predicted with certainty.\n\nAt the most, you can be sure of what will happen in the next few weeks. Long-term planning is guesswork. Long-term planning is not as relevant as it used to be because of faster technological change and increased competition in the market.\n\nIn today\u2019s dynamic environment, one needs to adapt quickly for survival and success.\n\nLong-term planning may still play a role in strategic decision-making for big firms with dominant positions in their market segment. But for us, lets\u2019 stick to a week.\n\nBefore planning your week, first, let\u2019s find out your budget. How much time do you really have? Here, I want to make a distinction. Whenever you think about time, always remember that time is not a single entity as far as scheduling your tasks.\n\nWhenever you think about time, think about a specific time and your energy level at that time. We want to find out time slots wherein you have high energy, medium energy, and low energy.\n\nSo even though you have got 24 hours a day, you have got a limited number of time-energy slots. When I say energy, it is physical as well as emotional energy. It is your overall state of mind and body. It is an indication of your resourcefulness which can predict the extent of your productivity.\n\nNow you have a meaningful and realistic inventory of time! Understand this, even if you have time, but if you are tired or are feeling sad, upset, or irritated, you can\u2019t really put that time to good use.\n\nNow, make a list of the goals you want to achieve this week, you can either write that on your note or at the 12-week year notion template.\n\nThen Exclude Sunday from the week. If you want you can have a five-day week.\nHere is my suggestion for choosing your goals for a week. Choose any five important categories of your life\u2014for example, business, fitness, family, learning, and personal.\n\nNow choose exactly one goal from each category that you want to achieve. So, you will have one business goal, one fitness goal, one family goal, one learning goal, and one personal goal. Remember, just one goal from each category.\n\nDon\u2019t be greedy; it\u2019s just for a week! Next week you can choose different goals if you want. And that\u2019s the beauty of this framework \u2013 freedom of making mistakes with no strings of guilt attached to your choice.\n\nIf you strictly follow this strategy, you will choose only the most important goals for a week. You will be ultra-focused. You will cut down the unnecessary crap. You will start saying no.\n\nImage description\n\nHere I assume that your commitment and sincerity are as high as a founder who wants to establish his vision to life!\n\nThis step will also reveal what your real priorities are. You just have a week; you have to choose what you want to do, you can\u2019t postpone essential things, you may not get another week! So, choose wisely.\n\nThe next step is to make a list of tasks needed to complete each goal. Then, sort the list into three categories: high energy tasks, medium energy tasks, and low energy tasks.\n\nNow you are clear about what to do and what is the best time to do it! That\u2019s it!!\n\nIf you want to be even more productive, you can select just one micro goal for a day and apply all of your energy to it. You need to avoid multitasking and just focus on one task at a time.\n\nThe side effect of using this framework is that you will realize it\u2019s high time you quit a few things that are so dear to you.\n\nIt will also help you recognize your actual responsibilities.\n\nFinally, it will also clear all the confusion about who you are and what\u2019s your purpose!\n\nI recently came across this informative post that I had saved in my Notion notes years ago. While going through my saved posts, I stumbled upon it again and read it twice.\n\nThis time, it struck a chord with me in a different way, and I felt a strong connection to it, so I decided to share it here by adding my perspective to it. All credit for the original post goes to Prashant Chaudhari, a wonderful writer and importantly a marketer with a vision.\n\nI tried to find more of his similar posts online, but it seems he has stopped writing about productivity topics, idk why but anyway enjoy the post:)"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a fun and engaging blog post about an unconventional productivity hack. Use a real example of someone publicly committing to a goal (e.g., live-streaming their startup development) and explain how external accountability and real-time engagement boosted productivity. Invite readers to share their own unusual productivity tricks."
            },
            {
                "role": "assistant",
                "content": "A weird productivity\nhack, that worked\nby Marc Lou\nHey Indie Hackers \ud83d\ude4c\n\n2 weeks ago, I started streaming my screen on Youtube with one goal in mind: Building a startup live in 7 days. \ud83c\udfa5\n\nstupid french guy building a startup live\n\nIt was stressful at first. I thought no one would tune in and the stream would be boring... \ud83d\ude14\n\nBut after a week (or 60 hours of streaming), the app was available on iOS, Android, and the web\n\nUPDATE: It got #1 badge from Product Hunt \ud83c\udf89\n\nAlso, totally unexpected, viewers debugged my code and redesigned the app \ud83e\udd2f\n\nEven if only 1 person is barely watching, you can't not show up or binge Netflix.\n\nCommit to a goal in public + Show ongoing progress = Productivity\n\nWhat is your weird productivity tip?"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a community-focused post announcing a curated list of over 100 productivity apps. Explain why the research was conducted, how the apps are categorized, and what key details (e.g., pricing, features, reviews) are included. Encourage readers to explore the collection and suggest additional tools for future updates."
            },
            {
                "role": "assistant",
                "content": "I curated a map of 100+\nproductivity apps (incl.\nscreenshots/pricing/rates\u2026)\nby Mogan Kung\nHey Indie hackers,\ud83d\udc4b\n\nI did some research and curated a list of over 100 productivity apps, from digital note-taking apps to project management tools, hoping to increase your productivity.\n\nWhy did I make it?\nI know I'm not the only one who is feeling overwhelmed by the never-ending to-do list and struggling to stay productive. There are a variety of productivity apps out there that can help, but finding and trying those apps can be time-consuming and exhausting. That's why I made this map, so you can find the app that best suits your needs in a few minutes.\n\nWhat information is included in this map?\nI categorize these productivity apps by their usage, like note-taking, calendar, project management, etc. So you can go directly to the category you need.\nFor each app, I summarize a general description, key features, screenshots, platforms supported, pricing, rates and reviews from G2 and Capterra for you to give a comprehensive evaluation and decide which app is the best for you.\n\nYou can have a glimpse of the map in this infographic:\nProductivity Tools Market Map\n\nOr, check out the article for detailed information of each app: https://gemoo.com/blog/best-productivity-apps.htm\n\nAlso, what's your favorite productivity tool that I didn't include? I will keep updating this map. (Next Step: add AI tools, what's your favorite?) Thanks for your ideas and want to know your feedback!"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a structured guide outlining practical productivity techniques. Cover key aspects such as optimizing your environment, using checklists, managing time effectively, prioritizing physical health, and taking rest when needed. Conclude with an FAQ addressing common productivity concerns"
            },
            {
                "role": "assistant",
                "content": "Practical Tips to Maintain Productivity\nby Martin Baun\n\nProductivity is a major driving force in my team. I have implemented tools and principles to help everyone in my team optimize their creativity. I have written articles detailing the same and now is the right time to complete the trilogy.\nLet\u2019s discuss some practical tips I use to keep myself productive from within.\n\nimgur.com\n\nUnderstanding your environment\nYour environment determines how productive you can be. I always prime my environment by removing any potential distractions from my workspace. I like focusing on one task and giving it my full attention. This means removing noise, loud music, programs, or entertainment sources. I keep my workspace simple with everything I need to accomplish my tasks. I sit at my computer and follow my planned checklist to accomplish my tasks.\n\nI do not use any productivity aids. These are things like white noise machines or timers to keep you accountable. I find them more of a distraction than an aid. Keeping the workspace lean with one visual and mental stimulus helps me maintain my productivity. Eliminating all distractions will keep you primed for the best output you can achieve. Your environment is one of the aspects to prime. Your workflow is the other. I do this by using checklists.\n\nUtilizing checklists\nChecklists are like to-do lists but better. They fully break down a task into specific deliverables ensuring every aspect of a task is accomplished as required. They help you maximize your time and get more done efficiently and correctly. I plan my days by deciding on the most important tasks and other smaller tasks that can be done during the day. These checklists keep me focused on the most important things, saving me time and energy.\n\nI've implemented these checklists in my remote team to great effect. I've seen my team reach peak productivity as they perform their tasks. We get things done and avoid the backlog, and wasted effort we previously experienced. The checklists are a staple of how I do things and how my team does things. To-do lists tell you what to do without breaking it down for you. It doesn't protect you from focusing on irrelevant aspects. Efficiency is the main priority and checklists help me and my team achieve this. I have written a detailed article that goes deeper into this topic. It will guide you on how to create, implement, and use these checklists.\n\nRead: Boost Productivity With Checklist\n\nProper time management\nI manage my time by planning my days. I have a simple text document that I use to write down a to-do list. I pick out the most important task to accomplish on the list and make it the most important adventure. I then decide which smaller tasks can fit my workday. These can range from meetings, giving, feedback, or tangible tasks I have to do. I prioritize the most important adventure as it is the main task with the biggest outcome for that day.\n\nI organize my workday to spend less time deciding what to do. It helps me know what's urgent and what isn't. The to-do list is important for batching the tasks. The checklists then help me execute the tasks most efficiently. This prevents me from having to multitask or have my attention pulled in many directions. I like grouping similar tasks and handling them in one go. An example of this is when I give feedback to my writer. I usually have the time block reserved for specific tasks. It keeps me productive and ensures things keep moving. We do this on Goleko. It allows us to manage projects better.\n\nHaving a clear plan of attack is something I'd recommend everyone to implement. It will help you maintain your output and keep you productive.\n\nMaintaining physical health\nA healthy mind and body are essential to maintaining your productivity. You cannot produce optimally when you're not at your best. Maintaining your health is vital to enhancing your productivity levels. You can't do much when sick, tired, or hurt. I promote my physical health by eating a lot of meat, training, and going for walks. I drink a lot of water and get some rest. This allows me to stave off sickness and fatigue in most instances. It's not a full-proof strategy but it works well most of the time.\n\nI keep my mind healthy by ingesting helpful and productive content. I read, practice coding, and learn to integrate new legacy frameworks. This keeps my mind eager, fresh, and active. Your mind is the biggest driving force of your motivation, creativity, and resilience. Keeping it productive is essential for your increased output. I am a big advocate of using coffee to get an extra boost. Caffeine is a good mental stimulant that helps you stay focused, energized, and productive. This is a template is used by many people to achieve great things. Take care of your mind and body and they'll take care of you.\n\nRead: Tips on starting a startup\n\nRemembering to rest\nRest is essential for your mind and body to recover. I struggle to know when to take some time to rest. I like maximizing my efforts, leading to overworking. I'm not a good judge of when to rest but I have some cues that help me. One of these cues is an inability to do my tasks well. This can be poor quality, poor efficiency, or both. This is a sign of reduced focus, intent, and drive.\n\nI take time to rest and let my systems recharge. It's recommended to have eight hours of sleep to recover. I don't regularly hit these numbers but I make sure to get enough sleep in. Your productivity is adversely affected when you aren't at your best. Take breaks and rest when you feel tired, slow, or overwhelmed. It is counterproductive to have your efficiency drop and produce lackluster work. Take time to rest, replenish your strength, and then work as best you can.\n\nTakeaways\nYou can enhance your productivity and execute all your plans. Properly manage your time, prioritize your physical health, understand your environment, and utilize some checklists. It will help you enhance your productivity and allow you to produce your best work. These are some of the strategies that work well for me. There are more techniques and strategies you can use to enhance your productivity. Remember to assess and adjust when possible. Keep your goal at the forefront and tailor your strategy toward it. You'll see your efficiency and productivity soar from it.\n\nFAQs\nDo productivity tools work?\nSome people swear by them and others don't. They do not work for me but that doesn't mean they won't for you. Try them and assess whether you're more or less productive with them.\n\nHow important is sleep to staying productive?\nSleep is necessary to keep fresh and re-energized. Take a break when necessary and let your mind and body recover to serve you better on your next tasks.\n\nWhat's the Pomodoro technique?\nThe Pomodoro Technique is a time management method developed by Francesco Cirillo in the late 1980s. It involves breaking work into intervals. The intervals were 25 minutes in length, separated by short breaks. The technique aims to improve productivity and focus by using short bursts of concentrated work, followed by rest periods.\n\nHow can I remain productive in project management?\nYou can use a project management tool to organize and delegate tasks to your team. Use checklists to break down the important tasks in the project and use your time well to fulfill said tasks. Implement timely feedback with proper and open communication. These principles should help you maintain high productivity in your project management.\n\nFor these and more thoughts, guides, and insights visit my blog at martinbaun.com\n\nYou can find Martin on X"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a SaaS-specific productivity guide for founders struggling with focus. Outline a simple three-step strategy: organizing the browser, setting boundaries, and taking scheduled breaks. Recommend specific tools that help optimize workflow, such as tab management extensions and the Pomodoro Technique."
            },
            {
                "role": "assistant",
                "content": "Saas founders! I've got a perfect 3-\nstep productivity strategy for you \ud83e\udd29\nby VicDevelopmently\n\ud83d\udc4b Hey SaaS founders\n\nYou already know how important it is to stay productive and focused. But with so many distractions on the web, it can be challenging to get things done.\n\nI was in the same place for the past few months and after seeing the MRR stuck for a few weeks, I knew something had to change. Always setting really high goals for the day and not managing to achieve them.\n\nHere is what I changed\n1. Organising my browser\nI know that most of you guys have 10+ tabs open at one time :) As a founder, you have to check twitter, PH, IH, email, but you also need 5 more tabs for your productivity and maybe even entertainment.\n\nI started using a browser extension tool called BrowserUnicorn, this allows me have more websites in the same tab. Now I have 1 tab with my notifications(Twitter, Email, PH, and others), 1 tab for my productivity(Notion, Calendly, Pomofocus) and 1 for entertainment and \u2615\ufe0f brakes ;) (Soundcloud, Youtube)\n\nHave a look at my browser now :)\nalt text for screen readers\n\n2. Set boundaries\nSet specific times for checking email or responding to customers, and stick to them. Avoid getting distracted by social media or other non-work-related sites.\n\n3. Take breaks\nTake regular breaks to avoid burnout and keep your energy levels up. Use tools like the Pomodoro Technique or Time Out to schedule your breaks and stay on track. This really helped my out and I used a super cool tool called Pomofocus\n\nalt text for screen readers\n\nThis month stats are looking better than ever, so I suggest you guys try to organise your browser to achieve better productivity in your working time :)\n\nHope this helps and see you next time \u270c\ufe0f"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a success story of an entrepreneur who took a big risk, relocated, and eventually built a profitable SaaS business. Cover their journey from financial struggles to financial independence, breaking down key milestones like launching a side hustle, quitting their job, and scaling revenue. Include financial details and lessons learned."
            },
            {
                "role": "assistant",
                "content": "26-year-old dropped everything to build a better life. 17 years later, he's making $29k MRR.\nBy James Fleischmann\nSeptember 26, 2024\nSorin Alupoaie, founder of Swifteq\n43-year-old Sorin Alupoaie left everything behind to move with his wife from Romania to Dublin. After a period of uncertainty, he upleveled his income, then quit his job to focus on his business, Swifteq.\n\nNow, he's making \u20ac26k MRR. Let's take a peek at his finances.\n\nTaking a chance on Ireland\nSeventeen years ago, Sorin and his wife moved from Romania to Ireland. It wasn't easy.\n\nThey did it for two reasons: escaping corruption and increasing their income potential. But until they both found jobs, which took a stressfully long time, it meant that they were living in a more expensive country with less income.\n\nNew country. No friends. No family. Little money. Living in a shared house. It was a big step down, both financially and otherwise.\n\nBut they kept their eyes on the prize. And with hard work and determination, they found jobs, and began to see the fruits of their labors.\n\nFinding independence\nThese days, Sorin is a solo founder. Fully independent. He does exactly what he loves to do every day \u2014 he builds useful software for customers.\n\nMoney brings independence. That's the purpose of it.\n\nHis company, Swifteq, builds apps that expand the Zendesk platform to help customer support teams automate and streamline their workflows, allowing them to focus on providing a stellar customer experience.\n\nIt started as a side hustle. But after a year, he decided he had the revenue \u2014 \u20ac5k MRR \u2014 and savings to quit his job. So he started working full time.\n\nSorin needed that cushion before taking the leap. He says he \"couldn't afford to\" put his family's financial security at risk for his \"entrepreneurial ambitions\".\n\nThere\u2019s a huge difference between indie hackers with families and those without. If you\u2019re on your own, you can work 16 hours a day, cut all your expenses, and move to a low-cost country. But when you have a family, their needs come first. Your business has to come second.\n\nBut he needn't have worried. In less than 6 months after going full time on Swifteq, it was supporting him and then some.\n\nSwifteq revenue\nSwifteq's revenue has more than doubled in the past year. Here's what he's bringing in:\n\nRevenue: \u20ac26k MRR\n\nFunding: Savings\n\nBusiness savings account: A year of salary\n\nPersonal savings account: \u20ac0\n\nFounder pay: \u20ac6.5k/mo gross (give or take)\n\nHe says his pay is high for a solo business, but Ireland is expensive and he has two kids. And anyway, he isn't into the idea that founders need to pay themselves as little as possible and reinvest every penny into the business.\n\nIf you can\u2019t earn enough money from your business to live a decent life, it\u2019s not worth it. Restraining yourself might work in the short term, but you\u2019re betting your present on a possibly better future. If things don\u2019t work out, you might look back and regret the times you said \u201cno\u201d to that well-deserved holiday with your family. \n\nDon't take cash injections\nSorin's a bootstrapper. His first startup took significant investment from an angel investor back in 2017, and he won't do that again.\n\nI believe this approach \u2014 along with the mindset that invested capital is necessary to build a business \u2014 had a profoundly negative impact on how I approached every decision in the business. It ultimately contributed to the business' failure.\n\nHe believes bootstrapping should be the default, as cash injections give the \"false impression that you're onto something\" when you're not.\n\nWorse, he says it leads to founders prioritizing investors over customers.\n\nInvest in your future\nAs far as personal investments, Sorin only invests in his private pension fund, where he regularly contributes \u20ac1k/mo. He's using a shares-based plan, with more risk and higher potential returns.\n\nAccording to Sorin, in Ireland, pension funds offer the most tax-efficient way to invest personal money. Personal income from other sources, such as dividends, rental income, or profit from investments, is heavily taxed \u2014 ranging from 33% to 52%.\n\nAdditionally, for business owners, investing into a pension fund allows them to allocate money before any tax is applied, providing significant tax savings.\n\nHe wants to have his future covered.\n\nI need to make sure I have enough money to live comfortably when I retire. Also, it\u2019s important to have a backup plan in case my entrepreneurial ventures don\u2019t work out as planned.\n\nSwifteq expenses\nHere are his monthly business expenses:\n\nSalary: \u20ac6.5k\n\nDev contracting: \u20ac3k\n\nContent marketing: \u20ac2.5k\n\nAWS: \u20ac1.2k\n\nStripe fees: \u20ac800\n\nAll revenue beyond this is saved so that he can reinvest it back into the business \u2014 he plans to do that soon. Mostly in marketing and product development.\n\nPersonal expenses\nHere are his monthly personal expenses:\n\nMortgage: \u20ac1.5k\n\nOther loans: \u20ac400\n\nGroceries: \u20ac1.6k\n\nOther (going out, hobbies, etc.): \u20ac500\n\nHe has about \u20ac200k left to pay on that mortgage, with an interest rate of 2.75%. Luckily, they bought the house during the 2007-2009 recession, when prices were low. He says prices have gone up by 40-50% since 2015.\n\nDon't count pennies\nAccording to Sorin, the best thing you can do for your business, from a financial perspective, is to prioritize your work based on the return on your time investment. It's not easy, but it's important.\n\nThe best way to save money is to spend it on things that matter.\n\nSorin isn't a budgeter. He says counting pennies is too stressful. He'd rather focus on bigger financial decisions and long-term goals without sweating the small stuff.\n\nYou can easily go too far and become overly controlling about how you spend your money. It\u2019s important to leave room for experimentation and trying new things.\n\nHe has no guilt spending money on things he likes. But he says his best purchases are always business books, startup podcasts, and startup conferences. These things inspire him.\n\nAnd he loves to travel to events and talks \u2014 the relationships he's built this way have been particularly valuable.\n\nAlways side hustle first\nLet's end with his best advice:\n\nAlways start on the side. Never quit your main job to start a new business, even if you have some initial savings or an investor. I made this mistake with my previous startup and it cost me dearly.\n\nHe says that rather than being motivating, financial pressure is an unnecessary stress.\n\nWhen you\u2019re stressed, your brain goes into fight-or-flight mode. You can\u2019t think straight, and you make poor decisions.\n\nYou can follow Sorin on X and Linkedin. And check out Swifteq.\n\nPlease note that the above are opinions. This is meant for informational purposes only. It is not intended to be financial advice."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a candid and relatable post about struggling with design as a developer. Present two main solutions\u2014improving the skill through practice or outsourcing design work\u2014and invite readers to weigh in on which approach is best. End with an open-ended question asking for advice from the community."
            },
            {
                "role": "assistant",
                "content": "I suck at design. What now?\nby Jayjen Lee\nI suck at design.\n\nI've been developing for the past 6 years now, and have worked at a FAANG company, multiple startups, and now in my own business.\n\nBut I realized I could never get a site to \"look good\" in the way that others are able to.\n\nI've even taken a few courses on UI/UX design and familiarized myself with Figma, but it just isn't clicking.\n\nI think there are two pieces of conventional wisdom here \u2014\n\nOption 1 - Get better.\nTo be fair, I've only taken one or two courses, and watched maybe a dozen Youtube videos.\n\nDesign takes practice, just like coding, so I could continue on and try to improve. Once I have this skill, I'll be even more powerful.\n\nOption 2 - Outsource.\nThis is the other obvious answer.\n\nIf design is my weak point, and I've tried pretty hard at it, maybe I should outsource it.\n\nMy time is valuable, and there are more efficient ways to use my limited resources. Double down on my strengths.\n\nWhat would you do, and what would you recommend?\n\nAnything I'm missing?"
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a technical deep-dive on why SQL is inefficient for complex data science tasks. Explain the pain points of SQL in terms of complexity, debugging, performance, and scalability. Introduce a modern automation-based approach to data processing that eliminates the need for manual SQL coding, making data science workflows more efficient."
            },
            {
                "role": "assistant",
                "content": "Freeing Data Scientists from Heavy Coding: A New Era of Automation\nby BrettHoffman\nSQL is wasting lives of data scientists\nSQL is difficult to write\nNearly all data scientists use SQL for data exploration and analysis. SQL appears deceptively simple and offers a certain degree of interactivity, making it a seemingly ideal choice for these purposes.\n\nTo perform filtering and grouping operations, for example, SQL just needs one line of code:\n\nselect id,name from T where id=1\nselect area,sum(amount) from T group by area\nBut this is only limited to simple cases. When the computation becomes complex, SQL code becomes complicated, too. For example, to count the longest consecutive rising days of each stock, the SQL statement is as follows:\n\nSELECT CODE, MAX(con_rise) AS longest_up_days\nFROM (\n    SELECT CODE, COUNT(*) AS con_rise\n    FROM (\n        SELECT CODE, DT, SUM(updown_flag) OVER (PARTITION BY CODE ORDER BY CODE, DT) AS no_up_days\n        FROM (\n            SELECT CODE, DT, \n                CASE WHEN CL > LAG(CL) OVER (PARTITION BY CODE ORDER BY CODE, DT)  THEN 0\n                ELSE 1 END AS updown_flag\n            FROM stock\n        )\n    )\n    GROUP BY CODE, no_up_days\n)\nGROUP BY CODE\nAnd to perform the commonly seen funnel analysis on user behavior data in e-commerce industries:\n\nWITH e1 AS (\n\tSELECT uid,1 AS step1, MIN(etime) AS t1\n\tFROM events\n\tWHERE etime>=end_date-14 AND etime<end_date AND etype='etype1'\n\tGROUP BY uid),\ne2 AS (\n\tSELECT uid,1 AS step2, MIN(e1.t1) as t1, MIN(e2.etime) AS t2\n\tFROM events AS e2 JOIN e1 ON e2.uid = e1.uid\n\tWHERE e2.etime>=end_date-14 AND e2.etime<end_date AND e2.etime>t1 AND e2.etime<t1+7 AND etype='etype2'\n\tGROUP BY uid),\ne3 as (\n\tSELECT uid,1 AS step3, MIN(e2.t1) as t1, MIN(e3.etime) AS t3\n\tFROM events AS e3 JOIN e2 ON e3.uid = e2.uid\n\tWHERE e3.etime>=end_date-14 AND e3.etime<end_date AND e3.etime>t2 AND e3.etime<t1+7 AND etype='etype3'\n\tGROUP BY uid)\nSELECT SUM(step1) AS step1, SUM(step2) AS step2, SUM(step3) AS step3\nFROM e1 LEFT JOIN e2 ON e1.uid = e2.uid LEFT JOIN e3 ON e2.uid = e3.uid\nBoth examples involve multilayer nested subqueries, which are difficult to understand and more difficult to write.\n\nThere are many similar computing tasks in real-world business scenarios. For example:\n\nFind players who score three times continuously in one minute;\n\nFind the number of users in 7 days who are active in three continuous days;\n\nCompute retention rate of every day\u2019s new users a business retains in the next day;\n\nCompute the growth rate of a stock on the date when the price is both greater than both the previous and the next five days;\n\n\u2026\n\nThese complex requirements often necessitate multi-step procedures and involve order-based operations. Their SQL implementations are extremely roundabout with a nearly one hundred lines of N-layered nested statement. Data scientists are wasting their lives in writing such SQL statements.\n\nSQL is difficult to debug\nComplicated SQL statements are very inconvenient to debug, such as the above commonly seen complex nested SQL statements. To debug them, we need to disassemble the statement and perform debugging layer by layer. Thedisassembly process involves modification of the SQL statement, making the whole debugging procedure very complicated.\n\nSQL debugging layers\n\nThis is because SQL does not have common debugging methods such as \u201cset breakpoint\u201d and \u201cstep over\u201d. And data scientists have to go to the trouble of disassembling statements in order to perform debugging, which is a waste of their lives.\n\nSQL has low performance\nThe query performance of SQL heavily depends on database optimizer. A well-designed database product is able to automatically adopt a more efficient algorithm (instead of executing the SQL statement literally), but often the optimization mechanism breaks down when facing complex computing logics.\n\nHere is a simple example. To fetch the top 10 from 100 million records, SQL has the following code:\n\nSELECT TOP 10 x FROM T ORDER BY x DESC\nThough this SQL statement contains the ORDER BY keyword, database optimizer will use a more efficient algorithm instead of performing the full sorting (because big data sorting is very slow).\n\nNow let\u2019s modify the task to find the top 10 from each group, and SQL has the following implementation:\n\nSELECT * FROM (\n SELECT *, ROW_NUMBER() OVER (PARTITION BY Area ORDER BY Amount DESC) rn\n FROM Orders )\nWHERE rn<=10\nThe implementation only gets a little complicated, but most database optimizers already become incapable of performing the optimization. They cannot guess the SQL statement\u2019s purpose but can only execute the literal logic written in the statement to perform the sorting (as there is ORDER BY keyword). As a result, performance is sharply decreased.\n\nThe SQL statements in real-world business scenarios are far more complicated than that in the example. It is rather common that database optimizer becomes ineffective, such as the above SQL statement handling funnel analysis. The statement involves repeated joins, which are difficult to write and extremely slow to execute.\n\nLow performance means waiting. For certain big data processing cases, waiting times are from a number of hours to even one day. And during the long waits data scientists\u2019 lives passed.\n\nSQL is closed\nSQL is the formal language used by databases. The closed databases make data processing difficult. Being closed here refers to the database requirement that data to be computed and processed by the database should be loaded into it in advance. The border between internal data and external data is clear.\n\nIn real-life businesses, data analysts often need to process data coming from the other sources, including text, Excel, application interface, web crawler, to name a few. Sometimes data coming from any of those sources is only temporarily used, but loading them into the database for each use consumes database space resources and the ETL process is time-consuming. And databases usually have constraints, and non-standardized data cannot be loaded into. This requires to first re-organize data, which needs both time and resources, and then write them to the database, which is also time-consuming (as database writes are very slow). It is during handling these peripheral data organization, loading and retrieval operations that the life of a data scientist is wasted.\n\nPython is also wasting lives of data scientists\nAs SQL has too many problems, data scientists seek its replacements, and Python is one of them.\n\nPython surpasses SQL in many aspects. It is easier to debug and more open, and supports procedural computation. However, Python also has its flaws.\n\nComplex computations are still hard to handle\nPandas, one of Python\u2019s third-party libraries, offers a rich collection of computing functions, which make certain computations simpler than their SQL counterparts. However, handling complex scenarios may still be challenging. For example, here is the code for finding the longest consecutive rising days for each stock mentioned earlier:\n\nimport pandas as pd\nstock_file = \"StockRecords.txt\"\nstock_info = pd.read_csv(stock_file,sep=\"\\t\")\nstock_info.sort_values(by=['CODE','DT'],inplace=True)\nstock_group = stock_info.groupby(by='CODE')\nstock_info['label'] = stock_info.groupby('CODE')['CL'].diff().fillna(0).le(0).astype(int).cumsum()\nmax_increase_days = {}\nfor code, group in stock_info.groupby('CODE'):\n\tmax_increase_days[code] = group.groupby('label').size().max() \u2013 1\nmax_rise_df = pd.DataFrame(list(max_increase_days.items()), columns=['CODE', 'max_increase_days'])\nThe Python code, which involves hard-coding with \u201cfor\u201d loop, is still cumbersome. This continues to waste lives of data scientists.\n\nInconvenient debugging functionalities\nPython has many IDEs and provides much-better-than-SQL debugging capabilities such as \u201cbreakpoint\u201d, which makes it unnecessary to disassemble the code.\n\nBut viewing intermediate values still mainly relies on the print method, which needs to be removed after debugging. This proves to be somewhat cumbersome.\n\nPython debugging\n\nIt takes longer to debug when the corresponding functionality is inconvenient to use, which results in a waste of time for data scientists.\n\nLow big data processing capability and performance\nPython almost does not have any big data processing capabilities. While the Pandas library can perform in-memory computations like sorting and filtering directly, it struggles with datasets larger than the available memory. In that case data needs to be segmented and processed segment by segment with hardcoding, and the code becomes very complicated.\n\nPython's parallelism is superficial. To harness multiple CPUs, complex multi-process parallelism is often required. But this is beyond the reach of most data scientists. Bing unable to code parallel processing, data scientists can only perform the slow serial computations and witness their time wasted.\n\nBoth SQL and Python are not satisfactory enough, then what can truly rescue data scientists?\n\nesProc SPL \u2013 the rescuer of data scientists\nesProc SPL! A very tool specifically designed for structured data processing.\n\nSPL is characterized by conciseness, ease of understanding and convenient debugging. It supports large-scale data processing and delivers high performance, fundamentally addressing limitations of SQL and Python.\n\nSimple to write\nSPL offers rich data types and computing class libraries and supports procedural computation, greatly simplifying the code of implementing complex computations. To find the longest consecutive rising days for each stock, for example, SPL has the following code:\nSPL code\n\nThe SPL code is much shorter and does not involve the loop statement. Both data read and write are not difficult.\n\nTo perform the ecommerce funnel analysis:\nSPL code for funnel analysis\n\nCompared with its SQL counterpart, the SPL code is more concise and versatile, and more conforms to the natural way of thinking. It can be used to handle the funnel analysis involving any number of steps. E-commerce funnel analysis with SPL is simpler and aligns more with natural thinking. This code can handle any-step funnel, offering greater simplicity and versatility compared to SQL.\n\nConvenient to debug\nSPL also offers comprehensive debugging capabilities, including \u201cSet breakpoint\u201d, \u201cRun to cursor\u201d, \u201cStep over\u201d, etc. The result of each step is displayed in real-time on the right side of the IDE. This is convenient as users do not need to split away each subquery or perform manual print any more.\nsimple debugging\n\nSupport for big data processing\nSPL supports big data processing, no matter whether the data can fit into the memory or not.\n\nIn-memory computation:\nSPL code for In-memory computation\n\nExternal memory computation:\nSPL code for External memory computation\n\nWe can see that the SPL code for the external memory computation is the same as that for the in-memory computation, requiring no additional workload.\n\nIt is easy and convenient to implement parallel processing in SPL. You just need to add an @m option to the serial computation code.\nparallel processing\n\nHigh performance\nIt is easy to write code with low amount of computational work and achieves faster execution in SPL. Take the previously mentioned topN problem as an example, SPL treats topN as a kind of aggregation, eliminating full sorting from the computing logic and achieving much faster speed. This SPL capability is inherent and does not rely on the assistance of optimizer.\n\nGet TopN from the whole set:\n\nOrders.groups(;top(10;-Amount))\nGet TopN from each group:\n\n Orders.groups(Area;top(10;-Amount))\nThe code of getting topN from each group and that from the entire set is basically the same. Both are straightforward and runs fast.\n\nSPL offers a wide range of high-performance algorithms, including:\n\nFor search: Binary search, sequence-number-based location, index-based search, batch search, etc.\n\nFor traversal: Cursor filtering, multipurpose traversal, multicursor, new understanding of aggregation, order-based grouping, application cursor, column-wise computing, etc.\n\nFor association: Foreign-key-based pre-association, foreign key numberization, alignment sequence, big dimension table search, one-sided heaping, order-based merge, association-based location, etc.\n\nFor cluster computing: cluster composite table, duplicate dimension table, dimension table segmentation, load balancing, etc.\n\nEquipped with these algorithms, SPL can achieve a dramatic skyrocketing in computational performance. Data scientists do not need to waste their lives in long waiting.\n\nOpen system\nSPL is naturally open. It can directly compute any data sources as long as it can access them, including data files such as CSV and Excel, various relational and non-relational databases, and multi-layer data such as JSON and XML, and thus can perform mixed computations.\n\nopenness of SPL\n\nWith the open system, data scientists can process data coming from various sources directly and efficiently, getting rid of the time spent in data organization and data import/export and thus increasing data processing efficiency.\n\nHigh portability and enterprise-friendly\nSPL also offers the proprietary file format that is high-performance and portable.\nproprietary file format\n\nIn contrast, Python lacks a proprietary storage solution. Text files are slow, and databases result in the loss of portability.\n\nSPL is enterprise-friendly because it is developed in pure Java. After data exploration and analysis, data scientists can integrate SPL with the application by embedding its jars into the latter, facilitating smooth transition from outside to inside.\n\nWith the convenient to use, easy to debug, high-performance and integration-friendly SPL, data scientists can be freed from the heavy coding work and devote more time to their own businesses.\n\nesProc SPL is an open-source tool, and its source code is available here."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a detailed comparison of Java, SQL, and modern alternatives for data processing in application development. Discuss the limitations of Java for data handling, the benefits of SQL, and the rise of Java-based data processing frameworks. Introduce an open-source solution like esProc SPL, explaining its advantages, including concise syntax, independent computing capability, seamless Java integration, and large-scale data processing. Provide examples comparing Java, SQL, and SPL implementations."
            },
            {
                "role": "assistant",
                "content": "Java Devs Rejoice! This Open-Source Secret Weapon Will Cut Your Coding Time in Half\nby BrettHoffman\nComputing dilemma in the application\nDevelopment and Framework, which should be given the higher priority?\nJava is the most commonly used programming language in application development. But writing code to process data in Java isn\u2019t simple. For example, below is the Java code for performing grouping & aggregation on two fields:\n\nMap<Integer, Map<String, Double>> summary = new HashMap<>();\n    for (Order order : orders) {\n        int year = order.orderDate.getYear();\n        String sellerId = order.sellerId;\n        double amount = order.amount;\n        Map<String, Double> salesMap = summary.get(year);\n        if (salesMap == null) {\n            salesMap = new HashMap<>();\n            summary.put(year, salesMap);\n        }\n        Double totalAmount = salesMap.get(sellerId);\n        if (totalAmount == null) {\n            totalAmount = 0.0;\n        }\n        salesMap.put(sellerId, totalAmount + amount);\n    }\n    for (Map.Entry<Integer, Map<String, Double>> entry : summary.entrySet()) {\n        int year = entry.getKey();\n        Map<String, Double> salesMap = entry.getValue();\n        System.out.println(\"Year: \" + year);\n        for (Map.Entry<String, Double> salesEntry : salesMap.entrySet()) {\n            String sellerId = salesEntry.getKey();\n            double totalAmount = salesEntry.getValue();\n            System.out.println(\"  Seller ID: \" + sellerId + \", Total Amount: \" + totalAmount);\n        }\n    }\nBy contrast, the SQL counterpart is much simpler. One GROUP BY clause is enough to close the computation.\n\nSELECT year(orderdate),sellerid,sum(amount) FROM orders GROUP BY year(orderDate),sellerid\nIndeed, early applications worked with the collaboration of Java and SQL. The business process was implemented in Java at the application side, and data was processed in SQL in the backend database. The framework was difficult to expand and migrate due to database limitations. This was very unfriendly to the contemporary applications. Moreover, on many occasions SQL was unavailable when there were no databases or cross-database computations were involved.\n\nIn view of this, later many applications began to adopt a fully Java-based framework, where databases only do simple read and write operations and business process and data processing are implemented in Java at the application side, particularly when microservices emerged. This way the application is decoupled from databases and gets good scalability and migratability, which helps gain framework advantages while having to face the Java development complexity mentioned previously.\n\nIt seems that we can only focus on one aspect \u2013 development or framework. To enjoy advantages of the Java framework, one must endure difficulty development; and to use SQL, one need to tolerate shortcomings of the framework. This creates a dilemma.\n\nThen what can we do?\nWhat about enhancing Java's data processing capabilities? This not only avoids SQL problems, but also overcomes Java shortcomings.\n\nActually, Java Stream/Kotlin/Scala are all trying to do so.\n\nStream\nThe Stream introduced in Java 8 added many data processing methods. Here is the Stream code for implementing the above computation:\n\nMap<Integer, Map<String, Double>> summary = orders.stream()\n            .collect(Collectors.groupingBy(\n                    order -> order.orderDate.getYear(),\n                    Collectors.groupingBy(\n                            order -> order.sellerId,\n                            Collectors.summingDouble(order -> order.amount)\n                    )\n            ));\n\n    summary.forEach((year, salesMap) -> {\n        System.out.println(\"Year: \" + year);\n        salesMap.forEach((sellerId, totalAmount) -> {\n            System.out.println(\"  Seller ID: \" + sellerId + \", Total Amount: \" + totalAmount);\n        });\n    });\nStream indeed simplifies the code in some degree. But overall, it is still cumbersome and far less concise than SQL.\n\nKotlin\nKotlin, which claimed to be more powerful, improved furtherly:\n\nval summary = orders\n        .groupBy { it.orderDate.year }\n        .mapValues { yearGroup ->\n            yearGroup.value\n                .groupBy { it.sellerId }\n                .mapValues { sellerGroup ->\n                    sellerGroup.value.sumOf { it.amount }\n                }\n        }\n    summary.forEach { (year, salesMap) ->\n        println(\"Year: $year\")\n        salesMap.forEach { (sellerId, totalAmount) ->\n            println(\"  Seller ID: $sellerId, Total Amount: $totalAmount\")\n        }\n    }\nThe Kotlin code is simpler, but the improvement is limited. There is still a big gap compared with SQL.\n\nScala\nThen there was Scala:\n\nval summary = orders\n        .groupBy(order => order.orderDate.getYear)\n        .mapValues(yearGroup =>\n            yearGroup\n                .groupBy(_.sellerId)\n                .mapValues(sellerGroup => sellerGroup.map(_.amount).sum)\n    )\n    summary.foreach { case (year, salesMap) =>\n        println(s\"Year: $year\")\n        salesMap.foreach { case (sellerId, totalAmount) =>\n            println(s\"  Seller ID: $sellerId, Total Amount: $totalAmount\")\n        }\n    }\nScala is a bit simpler than Kotlin, but still cannot be compared to SQL. In addition, Scala is too heavy and inconvenient to use.\n\nIn fact, these technologies are on the right path, though they are not perfect.\n\nCompiled languages are non-hot-swappable\nAdditionally, Java, being a compiled language, lacks support for hot swapping. Modifying code necessitates recompilation and redeployment, often requiring service restarts. This results in a suboptimal experience when facing frequent changes in requirements. In contrast, SQL has no problem in this regard.\n\ncompiled language\n\nJava development is complicated, and there are also shortcomings in the framework. SQL has difficulty in meeting the requirements for framework. The dilemma is difficult to solve. Is there any other way?\n\nThe ultimate solution \u2013 esProc SPL\nesProc SPL is a data processing language developed purely in Java. It has simple development and a flexible framework.\n\nConcise syntax\nLet\u2019s review the Java implementations for the above grouping and aggregation operation:\n\nJava implementations\n\nCompare with the Java code, the SPL code is much more concise:\n\nOrders.groups(year(orderdate),sellerid;sum(amount))\nIt is as simple as the SQL implementation:\n\nSELECT year(orderdate),sellerid,sum(amount) FROM orders GROUP BY year(orderDate),sellerid\nIn fact, SPL code is often simpler than its SQL counterpart. With support for order-based and procedural computations, SPL is better at performing complex computations. Consider this example: compute the maximum number of consecutive rising days of a stock. SQL needs the following three-layer nested statement, which is difficult to understand, not to mention write.\n\nselect max(continuousDays)-1\n  from (select count(*) continuousDays\n    from (select sum(changeSign) over(order by tradeDate) unRiseDays\n       from (select tradeDate,\n          case when closePrice>lag(closePrice) over(order by tradeDate)\n          then 0 else 1 end changeSign\n          from stock) )\ngroup by unRiseDays)\nSPL implements the computation with just one line of code. This is even much simpler than SQL code, not to mention the Java code.\n\nstock.sort(tradeDate).group@i(price&lt;price[-1]).max(~.len())\nComprehensive, independent computing capability\nSPL has table sequence \u2013 the specialized structured data object, and offers a rich computing class library based on table sequences to handle a variety of computations, including the commonly seen filtering, grouping, sorting, distinct and join, as shown below:\n\nOrders.sort(Amount) // Sorting\nOrders.select(Amount*Quantity>3000 && like(Client,\"*S*\")) // Filtering\nOrders.groups(Client; sum(Amount)) // Grouping\nOrders.id(Client) // Distinct\njoin(Orders:o,SellerId ; Employees:e,EId) // Join\n\u2026\u2026\nMore importantly, the SPL computing capability is independent of databases; it can function even without a database, which is unlike the ORM technology that requires translation into SQL for execution.\n\nEfficient and easy to use IDE\nBesides concise syntax, SPL also has a comprehensive development environment offering debugging functionalities, such as \u201cStep over\u201d and \u201cSet breakpoint\u201d, and very debugging-friendly WYSIWYG result viewing panel that lets users check result for each step in real time.\n\nSPL debugging\n\nSupport for large-scale data computing\nSPL supports processing large-scale data that can or cannot fit into the memory.\n\nIn-memory computation:\nIn-memory computation\n\nExternal memory computation:\nExternal memory computation\n\nWe can see that the SPL code of implementing an external memory computation and that of implementing an in-memory computation is basically the same, without extra computational load.\n\nIt is easy to implement parallelism in SPL. We just need to add @m option to the serial computing code. This is far simpler than the corresponding Java method.\nimplement parallelism in SPL\n\nSeamless integration into Java applications\nSPL is developed in Java, so it can work by embedding its JARs in the Java application. And the application executes or invokes the SPL script via the standard JDBC. This makes SPL very lightweight, and it can even run on Android.\n\nCall SPL code through JDBC:\n\n    Class.forName(\"com.esproc.jdbc.InternalDriver\");\n    con= DriverManager.getConnection(\"jdbc:esproc:local://\");\n    st =con.prepareCall(\"call SplScript(?)\");\n    st.setObject(1, \"A\");\n    st.execute();\n    ResultSet rs = st.getResultSet();\n    ResultSetMetaData rsmd = rs.getMetaData();\nAs it is lightweight and integration-friendly, SPL can be seamlessly integrated into mainstream Java frameworks, especially suitable for serving as a computing engine within microservice architectures.\n\nHighly open framework\nSPL\u2019s great openness enables it to directly connect to various types of data sources and perform real-time mixed computations, making it easy to handle computing scenarios where databases are unavailable or multiple/diverse databases are involved.\n\nopenness of SPL\n\nRegardless of the data source, SPL can read data from it and perform the mixed computation as long as it is accessible. Database and database, RESTful and file, JSON and database, anything is fine.\n\nDatabases:\ndatabase code\n\nRESTful and file:\nRESTful and file code\n\nJSON and database:\nJSON and database code\n\nInterpreted execution and hot-swapping\nSPL is an interpreted language that inherently supports hot swapping while power remains switched on. Modified code takes effect in real-time without requiring service restarts. This makes SPL well adapt to dynamic data processing requirements.\n\ninterpreted language SPL\n\nThis hot\u2014swapping capability enables independent computing modules with separate management, maintenance and operation, creating more flexible and convenient uses.\n\nSPL can significantly increase Java programmers\u2019 development efficiency while achieving framework advantages. It combines merits of both Java and SQL, and further simplifies code and elevates performance.\n\nesProc SPL is open-source and now available: Open Source Address."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Create a list of nine alternative platforms where SaaS founders can launch and promote their products, apart from Product Hunt. Provide monthly traffic data, domain authority (DA), and a short description for each. End with a call-to-action directing readers to a larger curated list of platforms to help them get their first 1,000 users."
            },
            {
                "role": "assistant",
                "content": "9 Product Hunt alternatives to Launch your SaaS \ud83c\udfc6\nby Karthik\nHey makers,\n\nI have created a list of 9 PH alternatives where you can list your saas and get more users.\n\nUneed - (Monthly Traffic: 91K Visitors, DA: 59)\nPeerlist - (Monthly Traffic: 199K Visitors, DA: 64)\nDev hunt - (Monthly Traffic: 62K Visitors, DA: 57)\nMicro launch - (Monthly Traffic: 79K Visitors, DA: 44)\nFazier - (Monthly Traffic: 17K Visitors, DA: 58)\nSaaS hub - (Monthly Traffic: 358K Visitors, DA: 72)\nCtrl alt cc - (Monthly Traffic: 16K Visitors, DA: 37)\nTwelve Tools - (Monthly Traffic: 500 Visitors, DA: 16)\nPItchwall - (Monthly Traffic: 16K Visitors, DA: 65)\nFor more places like this, check out listmysaas.com. I have curated 150+ directories and platforms to list a saas and get the first 1000 users."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a news-style update about Reddit\u2019s new feature, Pro Trends, launched under Reddit Pro. Explain how this tool allows businesses to track trending topics, keywords, and brand mentions in real time. Discuss why this is important for marketers and how it can be used for organic growth strategies on Reddit."
            },
            {
                "role": "assistant",
                "content": "Reddit introduces a new tool to spot trends. Back in March 2024, Reddit introduced Reddit Pro, a free tool to \"help your business show up on Reddit\". This phrasing comes directly from their Pro landing page (below).\n\nYesterday, Reddit introduced Pro Trends, \"a new tool that allows businesses to uncover what people are saying about virtually any topic on Reddit\".\n\nWhy we care: This is what Reddit says: \"Now, with Reddit Pro Trends, businesses now can track virtually any keyword or phrase \u2013 their brand name, product name, category, or even the latest viral trend \u2013 and see when, where, and how redditors are discussing it in real time.\"\n\nSo if you have a product and want to do organic marketing on Reddit, this is your chance to start."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write an investigative-style article analyzing why many indie hackers are abandoning the 'build in public' approach. Discuss the early benefits of transparency, such as community support, investor interest, and personal branding. Then, explore the downsides, including copycats, competitive disadvantages, and security risks. Provide real-world examples of founders who have gone 'ghost mode' and suggest a phased approach for balancing transparency with business protection."
            },
            {
                "role": "assistant",
                "content": "Is this the end of \u201cBuild in Public\u201d? Here's why top indie hackers are suddenly disappearing.\n\"Build in public\" made millionaires. Now it's making founders targets. Here's why indie hackers are going ghost mode.\n\nBy Michal Kankowski\nFebruary 13, 2025\nA meme of Homer Simpson backing into bushes to hide\nIndie hackers used to share everything, but now many go private to protect their business.\n\nCopycats pop up at $10K+ MRR, but most fail. Still, they cause stress and take a share of your revenue.\n\nShare early, hide later. Be open as you grow, then go private at $10K+ MRR, sharing only key milestones at $30K+.\n\nOver the past two years, a quiet shift has rippled through the indie hacking community: founders who once championed radical transparency are now going dark. They\u2019re deleting MRR updates, scrubbing product URLs from bios, and retreating into stealth mode \u2014 all to protect their business from copycats.\n\nHere's why this trend is picking up speed, when it makes sense to jump on board, and how to stay open without leaving yourself exposed.\n\nThe rise of build in public movement\nThe \u201cbuild in public\u201d movement wasn\u2019t just a trend \u2014 it was a revolution.\n\nAs Damon Chen pointed out on his blog \u2014 Buffer and Ghost started the whole thing, inspiring thousands. By sharing their journey, they turned customers into die-hard fans, and it worked for years. Then came the golden era of building in public, with more and more creators jumping in and starting their own indie hacking adventures.\n\nFor years, indie hackers shared their product updates, revenue stats, and growth charts, creating a transparent environment that inspired others. Besides building a tight-knit community, let\u2019s be real \u2014 every revenue post grabbed a ton of attention and money that comes with it.\n\nSabba Keynejad, used it to scale VEED.IO to $500K MRR. The build in public movement has clear upsides. As he puts it:\n\nBenefit 1: Community\n\nWhen your first start, things can get lonely. The Indie Hackers community has been an incredible support network and inspiration for us. We have made great friends who have also supported and inspired us through our ups and downs \u2764\ufe0f\n\nBenefit 2: Investors\n\nDue to sharing revenue, we have had over 100 investors reach out to us, including most top-tier funds. Regardless if you are looking for capital or not, having an open dialogue with investors is no bad thing.\n\nBenefit 3: Hiring Talent\n\nWe have hired and also acquired other indie hacker companies (more on this coming soon). Being public is a great opportunity to tell your story.\nMany new hires mention how much they loved reading about our journey on the blog.\n\nBenefit 4: Personal Brand\n\nI have been invited onto many podcasts and had the opportunity to talk at conferences too. I have grown my Twitter following to 5K. The majority of my most popular Tweets have been about revenue milestones and growth\n\nThe copycat economy\nBut there\u2019s a dark side to transparency. As the indie hacking space has exploded, so has the number of copycats. Once you start gaining traction and being open about it, you\u2019re basically waving a flag saying you\u2019ve hit product-market fit. For a lot of people, this is the perfect chance to snag a proven idea without spending a dime or lifting a finger.\n\nThe best examples \u2014 Marc Lou, John Rush and the latest MVP Agency craze. Marc killed it with his boilerplate, and John \u2014 yeah, he\u2019s pretty much the \u201cDirectory Messiah\u201d now. And MVP Agencies? People started raking in $10K-$20K in just the first few months.\n\nDid people copy them? Oh boy, you bet! So many clones popped up that we ended up with directories of boilerplates, directories of MVP agencies, and even directories of directories.\n\nAs Danny Postma puts it:\n\ni think i actually never shared why i stopped sharing revenue numbers so here it is \n\nat a certain point: \n- sharing revenue becomes bragging \n- competitive disadvantage \n- personal safety due to bad actors \n- negatively influences relationships \n- doesn\u2019t tell the full story \n\nat a given moment the downsides become bigger than the upsides\n\nWe've had a bunch of successful indie hackers disappear into revenue ghost mode because of this. Just look at Damon Chen, Sabba Keynejad, Danny Postma, Levelsio, and the latest one \u2014 Jon Yongfook.\n\nEntry barriers\nThe problem with copycats is especially relevant for businesses with low barriers to entry. Seems obvious, but the easier it is to enter a market, the faster copycats swarm. The easier it is to replicate a business model, the easier target you become.\n\nThis is what Francesco Di Lorenzo, Typefully co-founder, said about his stance on getting back to sharing MRR updates:\n\nHere's why we stopped and why we're resuming it:\n\n1) Copycats pop up like crazy when your MRR is growing.\n\nWhy we stopped: you have to defend your business the best you can while you're still growing and consolidating it. Continuing to publicly share our MRR didn't help that.\n\nWhy we're resuming: copycats are no longer an issue because we have a very solid product and business + there's a 42k/mo entry price now. \ud83d\ude05\n\nWhen to go ghost mode\nWhen you're just starting out, sharing everything can and will be a huge boost. It\u2019s motivating, and the support is real. But once you hit a steady revenue the game changes. You've proven your business concept and now you're on the copycats' radar. It might not be the question \"if\", but rather \"when.\"\n\nSo when should you stop sharing? Alexander Belogubov suggests a clear threshold:\n\nWhy $10K?\n\nCopycat tipping point: Below $10K, your project is \u201ctoo small\u201d for clones. Above it, you\u2019re a target.\n\nMental health: As Jon Yongfook pointed out, publicly sharing revenue dip amplifies stress, \u201cwhen your numbers are live for the world to see, the level of stress and dread is amplified 10x.\u201d\n\nSafety: Tony Dinh once mentioned he might stop sharing his revenue because it could be risky for his business and personal safety. Levelsio removed the open revenue section from his site because of security concerns.\n\nCopycats gonna copy, but most will flop anyway\nYou might think the era of building in public is over, but we're far from it. You either succeed or learn. Copycats skip the learning part \u2014 and they rarely ever succeed. There are almost no cases of copycats outshining the original creator of a project.\n\nThey haven\u2019t mastered the subtle art of building a successful product \u2014 constant tweaks, branding, and marketing. Which ultimately forms your unique moat. They bypass all of that and jump straight to launching a copy-pasted, half-baked website or app. That\u2019s when the real learning curve hits them.\n\nSure, it\u2019s frustrating to see them take a share of the revenue that could have been yours. But you\u2019re here for the long game. Most of them don\u2019t have the patience to build something truly sustainable.\n\nThe new playbook: Build in public, grow in private\nThis isn\u2019t about abandoning transparency \u2014 it\u2019s about evolving it. That said, being open about your revenue can do wonders for the Indie Hackers community.\n\nMarc still drops his monthly numbers, and it\u2019s no surprise he\u2019s one of the most well-known names in the game.\n\nWhen Jon Yongfook asked if he should keep sharing his earnings, 60% of indie hackers were all for it.\n\nSo though the cons of building in public are pretty obvious, it still makes sense to do it, either for your own transparency or for the new generations of aspiring indie hackers.\n\nBut if you want to do it, here's the right way to go about it:\n\nPhase 1: Build in Public (0\u2013$10K MRR)\n\nShare progress to attract early users, transparency, and accountability.\n\nPhase 2: Go ghost mode ($10K+ MRR)\n\nDelete revenue stats, but keep sharing non-metric wins (e.g., customer stories, product lessons).\n\nProtect your moat: unknown features, key insights, best marketing strategies, etc.\n\nPhase 3: Selective transparency ($30K+ MRR)\n\nShare milestones sparingly (e.g., annual recaps).\n\nThe bottom line\nBuilding in public isn\u2019t dead \u2014 it\u2019s just getting a reality check.\n\nSharing your journey is great for getting noticed, building a community, and even landing investors. But once you hit real money, be ready for the copycats to show up.\n\nBe open when you\u2019re small, go quiet when you\u2019re scaling, and share just enough to stay relevant without giving away the secret sauce."
            }
        ]
    },
    {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Write a guide on financial survival strategies for bootstrapped founders. Cover topics like calculating a proper financial runway (6\u201324 months), the importance of location-based cost savings, and the debate between quitting a job vs. building a startup on the side. Include advice from successful indie founders on managing stress, avoiding financial burnout, and using freelancing as a funding source while maintaining focus on their startup."
            },
            {
                "role": "assistant",
                "content": "\u201cI have 5 months of runway left.\u201d Successful founders weigh in on how to bootstrap startups without burning through savings.\nIs pouring your personal funds into your project a savvy move or a setup for burnout? Here's a roundup of what successful founders say.\n\nBy Michal Kankowski\nFebruary 4, 2025\n\nAim for at least 6\u201324 months of living expenses to keep the lights on while you build and pivot.\n\nThe longer you can go without stressing over rent or groceries, the better your chances of finding product-market fit.\n\nYour savings stretch further in low-cost areas, so consider relocating to maximize your runway and focus.\n\nDo you keep your day job while grinding your startup on the side? Or do you burn through your own savings to give my business its best chance of success?\n\nIt's not just a hypothetical question. Well-known \"build-in-public\" founders like Dagobert Renouf open up about the topic on a regular basis:\n\nThe safety net magic number\nBefore you take the leap into full-time indie hacking, one of the biggest questions to resolve is how much runway you'll need.\n\nA good rule of thumb is to save at least 6\u201324 months\u2019 worth of living expenses. That gives you enough breathing room to experiment, pivot, and figure out what works without constantly sweating over rent or groceries.\n\nOf course, the exact number depends on your living situation and location.\n\nNot every spot on the planet is the same. $10,000 in Silicon Valley (famous for its crazy high living costs) will barely get you by for a month. But that same cash can let you live comfortably for like 10 months in Southeast Asia. It's also much easier to reach ramen profitability with low costs.\n\nThere are plenty of indie hackers backing up this approach:\n\nSure, it's way tougher if you've already got a family and kids. But if you're young and single, this might just be the way to go.\n\nYou\u2019ve gotta choose your hard.\n\nBurning your ships and going all in\nLeaving your cushy job to dive head-first into indie hacking is like pulling a Hern\u00e1n Cort\u00e9s \u2014 burning your boats so there\u2019s no turning back. Only, in your case, it\u2019s not galleons going up in flames, but the safety net of a steady paycheck. You\u2019re all-in, putting everything on the line to chase your dream.\n\nIt\u2019s the ultimate power move. No Plan B here. Just you, your idea, and all your energy laser-focused on making it happen. But if you pivot a bunch of times and still don\u2019t nail product-market fit, that \u201cno turning back\u201d mentality turns into a crash course in resilience.\n\nMental health risks of burning through your savings\nUsing up your life savings for months or even years on a startup, without any solid signs that it\u2019s about to take off, can mess with your head. And then with your health.\n\nBut once you\u2019ve burned through half your savings, you start running into this neighbor almost every time you go out. You know his name, his wife\u2019s and kid\u2019s names, and even some of his hobbies.\n\nWhen you\u2019re down to just a quarter of your savings, this neighbor moves in with you with his whole family and furniture. And the sh*t gets real.\n\nAnd when you get to the point of getting into debt, the stress can mess with your mental state really hard.\n\nHigh emotional states vs low emotional states\nIt's very important to realize how to manage your emotions while you're on your indie hacking journey.\n\nWhen you're feeling great, your brain's in an \"abundance state\" \u2014 everything feels doable, you act and react quickly, and you can figure out a fix to any problem.\n\nBut when you're in a bad mood, it's a \"lack state.\" Your mind feels cramped (some studies even say stress can lower your IQ), and you can't find answers to even the simplest issues. The worse you feel, the fewer options you see. At rock bottom, there's basically only one way out\u2026\n\nHaving a solid emergency fund or a side gig helps maintain a stable, resourceful mind.\n\nFreelancing your way to great mental health\nIf you\u2019ve already blown through a good chunk of your savings and the stress is starting to hit, freelancing might just be your lifesaver. It\u2019s not about ditching your dream \u2014 it\u2019s about using your already-obtained skills to get some cash, and keeping your head above water.\n\nA little extra income can seriously quiet that \u201chow the heck am I gonna pay rent in 3 months?\u201d voice in your head. Plus, freelancing gives you a bit of space from obsessing over your project. When inspiration strikes again, you\u2019ll be ready to crush it.\n\nKey takeaways\nSave at least 6\u201324 months\u2019 worth of living expenses before going all-in on indie hacking.\n\nYour savings runway depends on where you live \u2014 $10K stretches much further in Southeast Asia than in the EU or the US.\n\nFreelancing on the side can keep cash coming in while you work on your big idea.\n\nBurnout is real. Don\u2019t underestimate the mental toll of running out of money.\n\nThe ideal strategy is to build your side project to a point where it\u2019s making money before quitting your job.\n\nThe path of an indie hacker\nIndie hacking is not a walk in the park. It's full of big risks and even bigger mistakes. Whether you\u2019re burning your ships or building a slow-and-steady escape route, having a financial cushion can make all the difference.\n\nSave enough runway to buy yourself time, energy, and peace of mind. The journey to building something great isn\u2019t a sprint, it\u2019s a marathon. Keep hacking, keep learning, and remember: it\u2019s not just about surviving \u2014 it\u2019s about creating the life you\u2019ve always wanted."
            }
        ]
    }
] # type: ignore
    temp_data = {}
    for i in range(0, len(data)):
      temp_data.update{
          'prompt' : [
              data[i]['messages'][0],
              data[i]['messages'][1]
          ],
          'answer': data[i][2]
      }

    data = temp_data

    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

AttributeError: 'list' object has no attribute 'map'

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 83,886,080


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
</reasoning>Assume Ahmed's grade in the remaining 9 assignments is x. Emily's total grade in the first 9 assignments is 92. We can represent this as 92 + the grade on the final assignment. Since the final assignment is worth the same as the other assignments, we can say the grade on the final assignment is 0.9x, since it is 90% of the total of the other assignments. Now, we can write the equation for the total grades as follows: 

92 + 0.9x = 1x + 90 

Subtracting 92 from both sides gives us 0.9x = x - 2. 
Subtracting 0.9x from both sides gives us 0 = 0.1x - 2. Subse

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.000000,0.000000,196.500000,0.000000
2,0.000000,0.040667,0.099613,183.500000,0.000000
3,0.000000,-0.019833,0.035000,137.500000,0.000005
4,0.000000,0.409833,1.075630,188.000000,0.000007
5,0.000000,0.020333,0.049806,198.000000,0.000004
6,0.000000,0.000833,0.114188,140.666672,0.000006
7,0.000000,-0.028000,0.068586,140.833344,0.000009
8,0.000000,0.400833,1.029079,139.000000,0.000007
9,0.000000,0.000000,0.000000,184.333344,0.000009
10,0.000000,0.000000,0.000000,144.000000,0.000011


-------------------- Question:
The gauge on a water tank shows that the tank is 1/3 full of water. To fill the tank, 16 gallons of water are added. How many gallons of water does the tank hold when full? 
Answer:
24 
Response:
Let's break this problem down. 

If the tank is 1/3 full and 16 gallons are added to fill it, then the amount of water added is equal to the amount that was already in the tank (1/3 of the total capacity) plus the amount needed to fill it (2/3 of the total capacity). 

Let x be the capacity of the tank in gallons. 

The equation becomes x - (1/3)x + 16 = x.

To simplify the equation, we can combine the x terms: (2/3)x + 16 = x.

Next, subtract (2/3)x from both sides of the equation: (1/3)x = 16.

Now, multiply both sides of the equation by 3 to get rid of the fraction: x = 16 * 3

x = 48

The tank holds 48 gallons of water when full. 
Extracted:
Let's break this problem down. 

If the tank is 1/3 full and 16 gallons are added to fill it, then the amount of water 

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.000000,0.000000,196.500000,0.000000
2,0.000000,0.040667,0.099613,183.500000,0.000000
3,0.000000,-0.019833,0.035000,137.500000,0.000005
4,0.000000,0.409833,1.075630,188.000000,0.000007
5,0.000000,0.020333,0.049806,198.000000,0.000004
6,0.000000,0.000833,0.114188,140.666672,0.000006
7,0.000000,-0.028000,0.068586,140.833344,0.000009
8,0.000000,0.400833,1.029079,139.000000,0.000007
9,0.000000,0.000000,0.000000,184.333344,0.000009
10,0.000000,0.000000,0.000000,144.000000,0.000011


-------------------- Question:
Felix is chopping down trees in his backyard. For every 13 trees he chops down he needs to get his axe resharpened. It cost him $5 to sharpen his axe. If he spends $35 on axe sharpening, at least how many trees has he chopped down? 
Answer:
91 
Response:
Let's start by finding the number of times Felix got his axe sharpened. Since sharpening costs $5 each, and he spent $35 in total, he got his axe sharpened $35 / $5 = 7 times.

If for every 13 trees he chops down, he needs to get his axe sharpened, then after 7 sharpenings, he must have chopped down 7 * 13 = 91 trees.

<answer>91</answer> 
Extracted:
91
-------------------- Question:
Laura wants to bake a cake for her mother. She checks the recipe and the pantry and sees that she needs to buy 2 cups of flour, 2 cups of sugar, a cup of butter, and two eggs. The flour costs $4. The sugar costs $2. The eggs cost $.5, and the butter costs $2.5. When she is done baking it, she cuts the cake into 6 slices. Her 

TrainOutput(global_step=250, training_loss=0.001432442097942186, metrics={'train_runtime': 11046.913, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.023, 'total_flos': 0.0, 'train_loss': 0.001432442097942186})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.78s/it, est. speed input: 1.64 toks/s, output: 19.94 toks/s]


'Calculating pi to a large number of decimal places is a complex task that requires a computational approach, rather than a simple mathematical formula. Here\'s a way to calculate pi using the Monte Carlo method, which is an approximation method that uses random numbers to estimate the value of pi:\n\n**The Monte Carlo Method**\n\nThe Monte Carlo method is based on the idea of simulating the probability of a random walk across a square and circle. Here\'s the basic idea:\n\n1. Draw a square and a circle on a piece of paper.\n2. Generate random points within the square.\n3. Count the proportion of points that fall within the circle.\n4. The ratio of points within the circle to the total number of points is approximately equal to the ratio of the area of the circle to the area of the square, which is pi.\n\n**Mathematical Formulation**\n\nLet\'s denote the following variables:\n\n*   `N`: the number of random points generated\n*   `n`: the number of points within the circle\n*   `pi_appr

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.29s/it, est. speed input: 2.62 toks/s, output: 19.41 toks/s]


"<reasoning>\nPi (π) is an irrational number that represents the ratio of a circle's circumference to its diameter. It is approximately equal to 3.14159, but its decimal representation goes on indefinitely without repeating.\n\nTo calculate pi, we can use various mathematical formulas and methods, such as the Leibniz formula, the Gregory-Leibniz series, or the Monte Carlo method. However, these methods are not practical for obtaining a high degree of accuracy.\n\nA more practical approach is to use the Bailey-Borwein-Plouffe (BBP) formula, which is a spigot algorithm that allows us to calculate any digit of pi without having to compute the preceding digits.\n\nAnother method is to use the Chudnovsky algorithm, which is a fast and efficient method for calculating pi to a high degree of accuracy.\n\nFor simplicity, we can use the first few terms of the BBP formula to estimate pi:\nπ = 3 + 1/(4/3 - 1/(4/3 - 1/(4/3 - ...))\n\nLet's use this simplified formula to estimate pi:\n\nπ ≈ 3 + 1/(

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
